Rastrigin synthetic function:

GP ERM versus STP nu = 3 ERM (winner)

https://www.sfu.ca/~ssurjano/rastr.html

In [1]:
pip install pyGPGO


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.8 MB/s 
     |████████████████████████████████| 872 kB 54.3 MB/s 
  Created wheel for pyGPGO: filename=pyGPGO-0.5.1-py3-none-any.whl size=19880 sha256=3b869c6e4eacee55def677e203fcb3e744f6f8bf0ad365c9faaa76498ef3fafc
  Stored in directory: /root/.cache/pip/wheels/c8/5d/0b/2160114e2f1b87791c51b66cf07f89831dbb6f49167950316f
  Created wheel for Theano-PyMC: filename=Theano_PyMC-1.1.2-py3-none-any.whl size=1529964 sha256=a953908e4ab83fb5c2e875a9793a68e852620a1835ab8912bfaddb387bea4a4c
  Stored in directory: /root/.cache/pip/wheels/f3/af/8c/5dd7553522d74c52a7813806fc7ee1a9caa20a3f7c8fd850d5
Successfully built pyGPGO Theano-PyMC


In [2]:
### Import modules:

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import random
import warnings

from pyGPGO.logger import EventLogger
from pyGPGO.GPGO import GPGO
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.surrogates.tStudentProcess import tStudentProcess, logpdf
from pyGPGO.acquisition import Acquisition
from pyGPGO.covfunc import squaredExponential

from collections import OrderedDict
from joblib import Parallel, delayed
from numpy.linalg import slogdet, inv, cholesky, solve
from scipy.optimize import minimize
from scipy.spatial.distance import cdist
from scipy.special import gamma
from scipy.stats import norm, t
from matplotlib.pyplot import rc

rc('text', usetex=False)
plt.rcParams['text.latex.preamble']=[r'\usepackage{amsmath}']
plt.rcParams['text.latex.preamble'] = [r'\boldmath']

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
### Inputs:

obj_func = 'Rastrigin'
n_test = 500 # test points
eps = 1e-08
df = 3 # nu

util_gp = 'RegretMinimized'
util_stp = 'tRegretMinimized'
n_init = 5 # random initialisations

cov_func = squaredExponential()
hyperOpt = True


In [4]:
### Objective function:

if obj_func == 'Rastrigin':
    
    # True y bounds:
    y_lb = 0
    operator = -1 # targets global minimum 
    y_global_orig = y_lb * operator # targets global minimum
            
# Constraints:
    lb = -5.12 
    ub = +5.12

# Input array dimension(s):
    dim = 2

# 2-D inputs' parameter bounds:
    param = {'x1_training': ('cont', [lb, ub]),
             'x2_training': ('cont', [lb, ub])}
    
# Test data:
    x1_test = np.linspace(lb, ub, n_test)
    x2_test = np.linspace(lb, ub, n_test)
    Xstar_d = np.column_stack((x1_test, x2_test))

    def f_syn_polarity(x1_training, x2_training):
        return  operator * (10 * dim + x1_training** 2 - 10 * np.cos(2 * np.pi * x1_training)
                                    + x2_training** 2 - 10 * np.cos(2 * np.pi * x2_training)
                          )
    

In [5]:
max_iter = dim * 10  # iterations of Bayesian optimisation


In [6]:
### Cumulative Regret Calculator:

def min_max_array(x):
    new_list = []
    for i, num in enumerate(x):
            new_list.append(np.min(x[0:i+1]))
    return new_list


In [7]:
### Set-seeds:

run_num_1 = 1
run_num_2 = 2
run_num_3 = 3
run_num_4 = 4
run_num_5 = 5
run_num_6 = 6
run_num_7 = 7
run_num_8 = 8
run_num_9 = 9
run_num_10 = 10
run_num_11 = 11
run_num_12 = 12
run_num_13 = 13
run_num_14 = 14
run_num_15 = 15
run_num_16 = 16
run_num_17 = 17
run_num_18 = 18
run_num_19 = 19
run_num_20 = 20


In [8]:
### Acquisition function - ERM:

class Acquisition_new(Acquisition):    
    def __init__(self, mode, eps=eps, **params):
        
        self.params = params
        self.eps = eps

        mode_dict = {
            'RegretMinimized': self.RegretMinimized,
            'tRegretMinimized': self.tRegretMinimized
        }

        self.f = mode_dict[mode]
   
    def RegretMinimized(self, tau, mean, std):
        
        z = (mean - y_global_orig - self.eps) / (std + self.eps)
        return z * (std + self.eps) * norm.cdf(z) + (std + self.eps) * norm.pdf(z)[0]
    
    def tRegretMinimized(self, tau, mean, std, nu=3.0):
        
        gamma = (mean - y_global_orig - self.eps) / (std + self.eps)
        return gamma * (std + self.eps) * t.cdf(gamma, df=nu) + (std + self.eps) * (nu + gamma ** 2)/(nu - 1) * t.pdf(gamma, df=nu)
    
    def eval(self, y_global_orig, mean, std):
    
        return self.f(y_global_orig, mean, std, **self.params)
        
    def t_eval(self, y_global_orig, mean, std, nu=df):
    
        return self.f(y_global_orig, mean, std, nu=df, **self.params)


In [9]:
n_start_AcqFunc = max_iter
inverse = -1


In [10]:
## GPGO_multi: Gaussian

n_start = n_start_AcqFunc

class GPGO_multi(GPGO):

    def __init__(self, surrogate, acquisition, f, parameter_dict, n_jobs=1):
        self.GP = surrogate
        self.A = acquisition
        self.f = f
        self.parameters = parameter_dict
        self.n_jobs = n_jobs

        self.parameter_key = list(parameter_dict.keys())
        self.parameter_value = list(parameter_dict.values())
        self.parameter_type = [p[0] for p in self.parameter_value]
        self.parameter_range = [p[1] for p in self.parameter_value]

        self.history = []
        self.header =   'Evaluation \t Proposed point \t  Current eval. \t  Best eval. \t Min. ApproxAcqFunc '
        self.template = '{:3}\t {}\t {:3}\t {:3}\t {:3}'
 
    def acqfuncApprox(self, xnew):
        new_mean, new_var = self.GP.predict(xnew, return_std=True)
        new_std = np.sqrt(new_var + eps)
        f = self.A.eval(self.tau, new_mean, new_std)

        return -f
   
    def _optimizeAcq(self, method='L-BFGS-B'):
        
        start_points_dict = [self._sampleParam() for i in range(n_start)]
        start_points_arr = np.array([list(s.values())
                                     for s in start_points_dict])
        x_best = np.empty((n_start, len(self.parameter_key)))
        f_best = np.empty((n_start,))
        opt = Parallel(n_jobs=self.n_jobs)(delayed(minimize)(self.acqfuncApprox,
                                                                 x0=start_point,
                                                                 method=method,
                                                                 jac = False,
                                                                 bounds=self.parameter_range) for start_point in
                                               start_points_arr)
        x_best = np.array([res.x for res in opt])
        f_best = np.array([np.atleast_1d(res.fun)[0] for res in opt])
        f_best_min = min(f_best)

        self.x_best = x_best
        self.f_best = f_best
        self.f_best_min = f_best_min
        self.best = x_best[np.argmin(f_best)]
        self.start_points_arr = start_points_arr        
        self.history.append(self.f_best_min)

    def _printInit(self):
        print(self.header)
        for init_eval in range(self.init_evals):
            print(self.template.format('init', self.GP.X[init_eval], inverse * self.GP.y[init_eval], inverse * self.tau, ''))
      
    def _printCurrent(self):
        OKGREEN = '\033[92m'
        ENDC = '\033[0m'
        BOLD = '\033[1m'
        eval = str(len(self.GP.y) - self.init_evals)
        proposed = str(self.best)
        curr_eval = str(inverse * self.GP.y[-1])
        curr_best = str(inverse * self.tau)
        min_acqfunc = str(self.f_best_min)
        if float(curr_eval) <= float(curr_best):
            eval = BOLD + OKGREEN + eval + ENDC
            proposed = BOLD + OKGREEN + proposed + ENDC
            curr_eval = BOLD + OKGREEN + curr_eval + ENDC
            curr_best = BOLD + OKGREEN + curr_best + ENDC
            min_acqfunc = BOLD + OKGREEN + min_acqfunc + ENDC
        print(self.template.format(eval, proposed, curr_eval, curr_best, min_acqfunc))
        
    def run(self, max_iter=10, init_evals=3, resume=False):
        
        if not resume:
            self.init_evals = init_evals
            self._firstRun(self.init_evals)
            self._printInit()
        for iteration in range(max_iter):
            self._optimizeAcq()
            self.updateGP()
            self._printCurrent()


In [11]:
## GPGO_multi_STP: Student's-t

class GPGO_multi_STP(GPGO):

    def __init__(self, surrogate, acquisition, f, parameter_dict, n_jobs=1):
        self.GP = surrogate
        self.A = acquisition
        self.f = f
        self.parameters = parameter_dict
        self.n_jobs = n_jobs

        self.parameter_key = list(parameter_dict.keys())
        self.parameter_value = list(parameter_dict.values())
        self.parameter_type = [p[0] for p in self.parameter_value]
        self.parameter_range = [p[1] for p in self.parameter_value]

        self.history = []
        self.header =   'Evaluation \t Proposed point \t  Current eval. \t  Best eval. \t Min. ApproxAcqFunc '
        self.template = '{:3}\t {}\t {:3}\t {:3}\t {:3}'
 
    def acqfuncApprox(self, xnew):
        new_mean, new_var = self.GP.predict(xnew, return_std=True)
        new_std = np.sqrt(new_var + eps)
        f = self.A.t_eval(self.tau, new_mean, new_std)

        return -f
   
    def _optimizeAcq(self, method='L-BFGS-B'):
        
        start_points_dict = [self._sampleParam() for i in range(n_start)]
        start_points_arr = np.array([list(s.values())
                                     for s in start_points_dict])
        x_best = np.empty((n_start, len(self.parameter_key)))
        f_best = np.empty((n_start,))
        opt = Parallel(n_jobs=self.n_jobs)(delayed(minimize)(self.acqfuncApprox,
                                                                 x0=start_point,
                                                                 method=method,
                                                                 jac = False,
                                                                 bounds=self.parameter_range) for start_point in
                                               start_points_arr)
        x_best = np.array([res.x for res in opt])
        f_best = np.array([np.atleast_1d(res.fun)[0] for res in opt])
        f_best_min = min(f_best)

        self.x_best = x_best
        self.f_best = f_best
        self.f_best_min = f_best_min
        self.best = x_best[np.argmin(f_best)]
        self.start_points_arr = start_points_arr        
        self.history.append(self.f_best_min)

    def _printInit(self):
        print(self.header)
        for init_eval in range(self.init_evals):
            print(self.template.format('init', self.GP.X[init_eval], inverse * self.GP.y[init_eval], inverse * self.tau, ''))
      
    def _printCurrent(self):
        OKGREEN = '\033[92m'
        ENDC = '\033[0m'
        BOLD = '\033[1m'
        eval = str(len(self.GP.y) - self.init_evals)
        proposed = str(self.best)
        curr_eval = str(inverse * self.GP.y[-1])
        curr_best = str(inverse * self.tau)
        min_acqfunc = str(self.f_best_min)
        if float(curr_eval) <= float(curr_best):
            eval = BOLD + OKGREEN + eval + ENDC
            proposed = BOLD + OKGREEN + proposed + ENDC
            curr_eval = BOLD + OKGREEN + curr_eval + ENDC
            curr_best = BOLD + OKGREEN + curr_best + ENDC
            min_acqfunc = BOLD + OKGREEN + min_acqfunc + ENDC
        print(self.template.format(eval, proposed, curr_eval, curr_best, min_acqfunc))
        
    def run(self, max_iter=10, init_evals=3, resume=False):
        
        if not resume:
            self.init_evals = init_evals
            self._firstRun(self.init_evals)
            self._printInit()
        for iteration in range(max_iter):
            self._optimizeAcq()
            self.updateGP()
            self._printCurrent()


In [12]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 1

np.random.seed(run_num_1)
surrogate_gp_1 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_1 = GPGO_multi(surrogate_gp_1, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_1.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-0.84969467  2.25612281]	 20.33436270766351	 19.908403246996286	    
init	 [-5.1188288  -2.02411446]	 33.07414982069084	 19.908403246996286	    
init	 [-3.61721968 -4.17445279]	 53.347974723929894	 19.908403246996286	    
init	 [-3.21269544 -1.58145816]	 39.218472310354045	 19.908403246996286	    
init	 [-1.05710106  0.39748336]	 19.908403246996286	 19.908403246996286	    
1  	 [ 4.04077223 -4.24914728]	 44.655863836006326	 19.908403246996286	 -0.7978845605763168
2  	 [4.64028793 5.12      ]	 66.81727560213392	 19.908403246996286	 -0.7978845386271473
3  	 [-5.12  5.12]	 57.849427451571785	 19.908403246996286	 -0.7978760222071729
4  	 [5.12       0.33809514]	 44.29585437783213	 19.908403246996286	 -0.7975552609791186
5  	 [ 0.22553114 -5.12      ]	 37.44420393522627	 19.908403246996286	 -0.787408603407545
6  	 [0.75631098 5.12      ]	 39.10029321304164	 19.908403246996286	 -0.7679907916733563
7  	 [ 

In [13]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 2

np.random.seed(run_num_2)
surrogate_gp_2 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_2 = GPGO_multi(surrogate_gp_2, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_2.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-0.6554122  -4.85451539]	 43.490296251903594	 18.964539418712707	    
init	 [ 0.50854377 -0.6622987 ]	 35.91861667536101	 18.964539418712707	    
init	 [-0.81543371 -1.73737143]	 20.479562046739524	 18.964539418712707	    
init	 [-3.02439799  1.2213347 ]	 18.964539418712707	 18.964539418712707	    
init	 [-2.05153614 -2.3876887 ]	 28.041315668371354	 18.964539418712707	    
1  	 [ 5.12 -5.12]	 57.849427451571785	 18.964539418712707	 -0.7978839545016416
2  	 [4.77484753 5.12      ]	 60.16900059908957	 18.964539418712707	 -0.7978845607368477
3  	 [-0.33018675  5.12      ]	 43.86155308849719	 18.964539418712707	 -0.7977883707022733
4  	 [5.12       0.08038346]	 30.179741015980827	 18.964539418712707	 -0.7976642061783215
5  	 [-5.12  5.12]	 57.849427451571785	 18.964539418712707	 -0.7975065501636394
6  	 [-5.12 -5.12]	 57.849427451571785	 18.964539418712707	 -0.7960279228818826
7  	 [2.21222073 2.420588

In [14]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 3

np.random.seed(run_num_3)
surrogate_gp_3 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_3 = GPGO_multi(surrogate_gp_3, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_3.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [0.52017052 2.1314337 ]	 27.953858411008774	 10.607662635789808	    
init	 [-2.14113547  0.11087468]	 10.607662635789808	 10.607662635789808	    
init	 [4.02377681 4.05804123]	 33.42749829480097	 10.607662635789808	    
init	 [-3.83400642 -2.99783293]	 28.650953928965198	 10.607662635789808	    
init	 [-4.59297584 -0.6061072 ]	 57.6631355589384	 10.607662635789808	    
1  	 [ 5.12 -5.12]	 57.849427451571785	 10.607662635789808	 -0.7978845607896432
2  	 [-4.58738203  5.09579813]	 67.29899005638403	 10.607662635789808	 -0.7978826589651652
3  	 [ 5.12       -0.40159085]	 47.23450659297393	 10.607662635789808	 -0.7974457543317222
4  	 [ 0.2907194 -5.12     ]	 41.53988614720974	 10.607662635789808	 -0.7976139524615257
5  	 [ 1.34928012 -1.5109385 ]	 39.92108250169747	 10.607662635789808	 -0.7760871870708544
6  	 [-0.77703748  5.12      ]	 37.83784551940017	 10.607662635789808	 -0.7522758946644962
7  	 [-2

In [15]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 4

np.random.seed(run_num_4)
surrogate_gp_4 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_4 = GPGO_multi(surrogate_gp_4, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_4.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [4.78238555 0.48365823]	 51.03163809010808	 14.323038259018315	    
init	 [4.84028785 2.19971578]	 39.78645699016559	 14.323038259018315	    
init	 [ 2.02474316 -2.90724357]	 14.323038259018315	 14.323038259018315	    
init	 [ 4.87705042 -5.05620219]	 52.80627247106233	 14.323038259018315	    
init	 [-2.52946061 -0.66773471]	 41.61497868486559	 14.323038259018315	    
1  	 [-5.12        4.34429436]	 63.38171584115068	 14.323038259018315	 -0.7978832869820007
2  	 [0.34209231 5.12      ]	 44.51054090468416	 14.323038259018315	 -0.7978757560500602
3  	 [-5.12 -5.12]	 57.849427451571785	 14.323038259018315	 -0.7978665514855415
4  	 [-0.95956488 -5.12      ]	 30.166482141127112	 14.323038259018315	 -0.7916353717853625
5	 [0.85281389 1.15155678]	 10.234974204732818	 10.234974204732818	 -0.7813544800100467
6  	 [3.88842839 5.12      ]	 46.40277499881027	 10.234974204732818	 -0.7039469225652909
7  	 [-2.0215

In [16]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 5

np.random.seed(run_num_5)
surrogate_gp_5 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_5 = GPGO_multi(surrogate_gp_5, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_5.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-2.84678993  3.79629882]	 33.93442008827236	 7.8108627039749745	    
init	 [-3.00319585  4.2865757 ]	 39.673876075575784	 7.8108627039749745	    
init	 [-0.11866943  1.14425716]	 7.8108627039749745	 7.8108627039749745	    
init	 [2.72289645 0.1886002 ]	 25.38160395721669	 7.8108627039749745	    
init	 [-2.08076286 -3.19773462]	 22.589982116319675	 7.8108627039749745	    
1  	 [ 5.12       -4.82933075]	 57.46650377631515	 7.8108627039749745	 -0.7978833409847951
2  	 [5.12 5.12]	 57.849427451571785	 7.8108627039749745	 -0.7978826840025247
3  	 [-5.12       -0.04624548]	 29.346042003871005	 7.8108627039749745	 -0.7971838244179176
4  	 [1.0036762 5.12     ]	 29.934747175161554	 7.8108627039749745	 -0.7914341618687212
5  	 [-5.12 -5.12]	 57.849427451571785	 7.8108627039749745	 -0.7891127647393021
6  	 [ 1.10697515 -5.12      ]	 32.32522730674058	 7.8108627039749745	 -0.7880161674108926
7  	 [ 5.12       

In [17]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 6

np.random.seed(run_num_6)
surrogate_gp_6 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_6 = GPGO_multi(surrogate_gp_6, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_6.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 4.02288795 -1.72052679]	 31.08835710146886	 17.28954482757088	    
init	 [ 3.28938622 -4.69302655]	 58.797867722203385	 17.28954482757088	    
init	 [-4.0175956   0.97333314]	 17.28954482757088	 17.28954482757088	    
init	 [ 0.30532979 -0.83141193]	 19.296253155889353	 17.28954482757088	    
init	 [-1.68542362  1.25459899]	 28.650630936276173	 17.28954482757088	    
1  	 [-3.50674745 -4.82185888]	 61.17543432012923	 17.28954482757088	 -0.79788320213269
2  	 [5.12 5.12]	 57.849427451571785	 17.28954482757088	 -0.7978845604970858
3  	 [-5.12  5.12]	 57.849427451571785	 17.28954482757088	 -0.7974614536659732
4  	 [0.39543742 5.12      ]	 46.999449818344814	 17.28954482757088	 -0.7972181321183911
5  	 [2.84230716 1.91250836]	 17.729625229985217	 17.28954482757088	 -0.7784530988606893
6  	 [-5.12       -1.80318523]	 38.89631343981903	 17.28954482757088	 -0.7099866693153976
7  	 [-0.1179019  -4.27594303

In [18]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 7

np.random.seed(run_num_7)
surrogate_gp_7 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_7 = GPGO_multi(surrogate_gp_7, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_7.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-4.33860312  2.86636843]	 45.646133072936244	 22.52235437888213	    
init	 [-0.63068947  2.28828342]	 34.83012662845338	 22.52235437888213	    
init	 [4.8946126  0.39419771]	 44.09657005662876	 22.52235437888213	    
init	 [ 0.01147355 -4.38219639]	 36.613209822404315	 22.52235437888213	    
init	 [-2.37118484e+00 -1.20319155e-03]	 22.52235437888213	 22.52235437888213	    
1  	 [-5.12 -5.12]	 57.849427451571785	 22.52235437888213	 -0.797870952716848
2  	 [ 5.12 -5.12]	 57.849427451571785	 22.52235437888213	 -0.7978668645922817
3  	 [3.59148976 5.12      ]	 60.21626667364799	 22.52235437888213	 -0.7977967392570915
4  	 [-4.02558214 -1.33646422]	 33.289738493146906	 22.52235437888213	 -0.7978845607896439
5	 [ 1.40596233 -0.96124902]	 21.500320538665335	 21.500320538665335	 -0.7700991234650246
6  	 [-1.80774683  5.12      ]	 38.643410349277715	 21.500320538665335	 -0.687967912205496
7  	 [2.33744294 2.

In [19]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 8

np.random.seed(run_num_8)
surrogate_gp_8 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_8 = GPGO_multi(surrogate_gp_8, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_8.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [3.82391708 4.79785639]	 50.20079446939181	 13.871821018360485	    
init	 [3.78055209 0.31596228]	 36.5114251593508	 13.871821018360485	    
init	 [-2.73686192 -5.00327624]	 43.34985765011677	 13.871821018360485	    
init	 [-0.7119993  -0.99992207]	 13.871821018360485	 13.871821018360485	    
init	 [ 0.23218863 -0.22126801]	 17.190590355445654	 13.871821018360485	    
1  	 [-3.08031965  4.1331065 ]	 31.115674086949312	 13.871821018360485	 -0.7978830989272913
2  	 [ 5.12 -5.12]	 57.849427451571785	 13.871821018360485	 -0.7978831350335036
3  	 [-5.12      -0.3998489]	 47.16917868555392	 13.871821018360485	 -0.7977020056075353
4  	 [ 1.18398229 -4.40957201]	 45.244896246982556	 13.871821018360485	 -0.7872465519327715
5  	 [0.42099732 3.28839966]	 42.173275721031196	 13.871821018360485	 -0.7659955685442256
6  	 [-2.38775987  1.21888673]	 32.85925876138888	 13.871821018360485	 -0.5457710428989525
7  	 [ 5

In [20]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 9

np.random.seed(run_num_9)
surrogate_gp_9 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_9 = GPGO_multi(surrogate_gp_9, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_9.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-5.01376866  0.01919582]	 25.248289026162446	 24.454800313488693	    
init	 [-0.04328148 -3.74958562]	 24.454800313488693	 24.454800313488693	    
init	 [-3.66478248 -2.88195916]	 39.46509426509438	 24.454800313488693	    
init	 [-0.83447623 -2.57944404]	 31.06766397812992	 24.454800313488693	    
init	 [-4.25922917 -1.58209393]	 49.922543556206975	 24.454800313488693	    
1  	 [3.69491889 4.82146316]	 55.95031066333101	 24.454800313488693	 -0.7978845607896435
2  	 [-2.1984441   5.08607262]	 38.945376636378526	 24.454800313488693	 -0.7978839100842573
3  	 [ 5.10449854 -0.6835496 ]	 42.65727260376921	 24.454800313488693	 -0.7978830981318622
4  	 [ 4.49589695 -5.12      ]	 69.13448023543316	 24.454800313488693	 -0.7973491299570542
5  	 [0.62500973 1.48418216]	 39.614721622579665	 24.454800313488693	 -0.7964725025894012
6  	 [-5.12       3.4091093]	 58.95990012960344	 24.454800313488693	 -0.74890962897

In [21]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 10

np.random.seed(run_num_10)
surrogate_gp_10 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_10 = GPGO_multi(surrogate_gp_10, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_10.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 2.77832339 -4.90750004]	 41.674330194390116	 13.839458310244165	    
init	 [1.36855793 2.54775176]	 44.69639719065837	 13.839458310244165	    
init	 [-0.01528819 -2.81808235]	 13.839458310244165	 13.839458310244165	    
init	 [-3.09183626  2.66783449]	 33.23221510904937	 13.839458310244165	    
init	 [-3.38830503 -4.2154003 ]	 54.73014366983691	 13.839458310244165	    
1  	 [ 5.12       -0.56860951]	 48.333154227419236	 13.839458310244165	 -0.7977527899905499
2  	 [5.12 5.12]	 57.849427451571785	 13.839458310244165	 -0.7975241622711899
3  	 [-5.12       -0.60617891]	 47.148100531475194	 13.839458310244165	 -0.7883923765029025
4  	 [-5.12  5.12]	 57.849427451571785	 13.839458310244165	 -0.7464575634575921
5  	 [-0.83793555  5.12      ]	 34.378549826358224	 13.839458310244165	 -0.7302606186127267
6	 [-1.23133442 -0.10555893]	 12.477244167931238	 12.477244167931238	 -0.7114547162651806
7  	 [ 1.896505

In [22]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 11

np.random.seed(run_num_11)
surrogate_gp_11 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_11 = GPGO_multi(surrogate_gp_11, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_11.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-3.27403839 -4.92057353]	 47.655641290890955	 10.679755252484755	    
init	 [-0.37664229  2.30332343]	 35.87889240695259	 10.679755252484755	    
init	 [-0.81711509 -0.14922651]	 10.679755252484755	 10.679755252484755	    
init	 [-4.98912446 -0.12931474]	 28.05462905574235	 10.679755252484755	    
init	 [4.52410012 3.59214172]	 71.62694632141611	 10.679755252484755	    
1  	 [ 3.73199281 -5.12      ]	 53.981496645775195	 10.679755252484755	 -0.7978845601917111
2  	 [-5.12  5.12]	 57.849427451571785	 10.679755252484755	 -0.7978751427330447
3  	 [ 5.12       -0.89340917]	 31.883000294632865	 10.679755252484755	 -0.7965638796638167
4  	 [ 0.31355448 -3.36749547]	 42.05621827157918	 10.679755252484755	 -0.7765707515784276
5  	 [1.20983407 5.12      ]	 37.89141684998465	 10.679755252484755	 -0.7289800108096495
6  	 [2.15080867 0.11826641]	 11.439478199953985	 10.679755252484755	 -0.6766706882833599
7  	 

In [23]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 12

np.random.seed(run_num_12)
surrogate_gp_12 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_12 = GPGO_multi(surrogate_gp_12, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_12.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-3.54137249  2.45810889]	 57.903242869085595	 40.51116653209555	    
init	 [-2.42365424  0.34549139]	 40.51116653209555	 40.51116653209555	    
init	 [-4.97075238  4.28796936]	 55.62655915398208	 40.51116653209555	    
init	 [ 4.10332011 -4.77776458]	 49.962803461970296	 40.51116653209555	    
init	 [ 4.6791612  -3.71497655]	 62.183891474990624	 40.51116653209555	    
1  	 [4.12801255 5.12      ]	 49.029236031330754	 40.51116653209555	 -0.7978845607895244
2  	 [-5.12 -5.12]	 57.84942745157172	 40.51116653209555	 -0.7978844770398728
3  	 [2.4087622  0.58849186]	 43.04348319114295	 40.51116653209555	 -0.7976096443302453
4	 [-0.48693949 -4.12693528]	 40.25048830391129	 40.25048830391129	 -0.7974025457415399
5  	 [-5.12       -1.54445326]	 50.92251384084567	 40.25048830391129	 -0.7311795299138888
6	 [-0.1736295  5.12     ]	 34.33839944237008	 34.33839944237008	 -0.7952243390173417
7	 [5.12       1.93248

In [24]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 13

np.random.seed(run_num_13)
surrogate_gp_13 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_13 = GPGO_multi(surrogate_gp_13, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_13.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 2.84367268 -2.68757791]	 33.58019830007169	 17.435826639425656	    
init	 [3.32061217 4.76927179]	 56.857057997503354	 17.435826639425656	    
init	 [ 4.83943541 -0.47667971]	 48.211919361679726	 17.435826639425656	    
init	 [1.11659482 2.82139151]	 17.435826639425656	 17.435826639425656	    
init	 [1.45012065 2.27346667]	 38.25352329493884	 17.435826639425656	    
1  	 [-4.10876704 -2.60547476]	 43.79941556238635	 17.435826639425656	 -0.7978845605124194
2  	 [-4.21031778  3.16136339]	 39.967696434876814	 17.435826639425656	 -0.7978829061025323
3  	 [-0.62129884 -5.12      ]	 46.544305319179244	 17.435826639425656	 -0.7957584444369071
4  	 [-0.72633081 -0.8250234 ]	 18.148708132481218	 17.435826639425656	 -0.7797128344843667
5  	 [-1.03400059  5.12      ]	 30.221197622661876	 17.435826639425656	 -0.7768349803895023
6  	 [-5.12        0.27690528]	 40.683858592773284	 17.435826639425656	 -0.60847818

In [25]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 14

np.random.seed(run_num_14)
surrogate_gp_14 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_14 = GPGO_multi(surrogate_gp_14, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_14.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [0.14277984 2.79721013]	 18.683085263052178	 10.423838604848608	    
init	 [ 3.7931795  -5.03759925]	 47.36348784446708	 10.423838604848608	    
init	 [-1.94830412  4.68586229]	 40.19779334078636	 10.423838604848608	    
init	 [ 0.13431513 -1.86076749]	 10.423838604848608	 10.423838604848608	    
init	 [ 0.40140736 -2.85434939]	 30.353548735049138	 10.423838604848608	    
1  	 [-4.70848678 -5.12      ]	 63.673437963575026	 10.423838604848608	 -0.7978833184109639
2  	 [5.12 5.12]	 57.849427451571785	 10.423838604848608	 -0.7978832735892897
3  	 [-5.12        0.35842555]	 45.35088849902042	 10.423838604848608	 -0.7978742288100157
4  	 [ 5.12       -0.06395315]	 29.725333604146282	 10.423838604848608	 -0.7978397022550058
5  	 [-1.64698364  0.04264381]	 19.101303442923523	 10.423838604848608	 -0.7256859867777973
6  	 [-5.12  5.12]	 57.849427451571785	 10.423838604848608	 -0.7396111794137876
7  	 [1.91878

In [26]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 15

np.random.seed(run_num_15)
surrogate_gp_15 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_15 = GPGO_multi(surrogate_gp_15, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_15.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 3.57189322 -3.28810573]	 54.938487770767075	 7.990765314336182	    
init	 [-4.56332069 -1.41784631]	 60.750198753157726	 7.990765314336182	    
init	 [-2.29989449  0.3072023 ]	 31.984997246800887	 7.990765314336182	    
init	 [-1.9873903  -2.00218256]	 7.990765314336182	 7.990765314336182	    
init	 [-3.97576933 -2.5610341 ]	 41.754957769694336	 7.990765314336182	    
1  	 [5.12       4.80077285]	 58.835817245851594	 7.990765314336182	 -0.7978845607896178
2  	 [-5.12        5.06616995]	 55.44268739247094	 7.990765314336182	 -0.7978827790068307
3  	 [0.00786909 5.12      ]	 28.936996176311574	 7.990765314336182	 -0.797822921469876
4  	 [2.25293295 1.11071713]	 18.817338560251102	 7.990765314336182	 -0.7968887159947444
5  	 [-0.33197529 -5.12      ]	 43.960843667147955	 7.990765314336182	 -0.7941092499069318
6  	 [5.12       0.08147383]	 30.213271943254803	 7.990765314336182	 -0.7394610337487173
7  	

In [27]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 16

np.random.seed(run_num_16)
surrogate_gp_16 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_16 = GPGO_multi(surrogate_gp_16, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_16.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-2.83349935  0.23719262]	 22.27210456874814	 22.27210456874814	    
init	 [ 0.51918292 -4.65303603]	 57.57021076017139	 22.27210456874814	    
init	 [-1.42613673 -2.83565116]	 33.89145899403749	 22.27210456874814	    
init	 [ 1.9325559  -3.44339021]	 35.85029586225333	 22.27210456874814	    
init	 [-4.39987336  4.51595121]	 77.78800881964571	 22.27210456874814	    
1  	 [2.142749   4.83374619]	 36.69381850109535	 22.27210456874814	 -0.7978845508179482
2  	 [5.12       0.59293433]	 47.61934452567999	 22.27210456874814	 -0.7978570248534114
3  	 [-5.12 -5.12]	 57.849427451571785	 22.27210456874814	 -0.7972211618053443
4	 [1.00686869 0.83549454]	 6.604011697570501	 6.604011697570501	 -0.7898772476407555
5  	 [-0.94229324  3.24295778]	 21.612520217652467	 6.604011697570501	 -0.7318586062739123
6  	 [ 5.12 -5.12]	 57.849427451571785	 6.604011697570501	 -0.7858159755426392
7  	 [-5.12       -1.65377298]	 4

In [28]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 17

np.random.seed(run_num_17)
surrogate_gp_17 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_17 = GPGO_multi(surrogate_gp_17, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_17.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-2.10263037  0.31320838]	 20.395145364684023	 20.395145364684023	    
init	 [-3.15882714 -4.42470033]	 53.03732051200137	 20.395145364684023	    
init	 [2.93873111 1.60085526]	 29.989224812583537	 20.395145364684023	    
init	 [1.40821398 0.77417363]	 29.451989415882437	 20.395145364684023	    
init	 [-4.71999574 -1.45598869]	 55.89242173757483	 20.395145364684023	    
1  	 [ 5.12 -5.12]	 57.849427451571785	 20.395145364684023	 -0.7978845604995516
2  	 [-4.70696616  5.12      ]	 63.751313255077804	 20.395145364684023	 -0.7978829227686138
3  	 [0.05624164 5.12      ]	 29.545781683187776	 20.395145364684023	 -0.7972639168284501
4  	 [5.12 5.12]	 57.849427451571785	 20.395145364684023	 -0.7965820621781331
5  	 [ 1.07726363 -3.5143271 ]	 34.62590542363461	 20.395145364684023	 -0.7949936509888714
6  	 [ 5.12       -1.25380509]	 40.73579906584446	 20.395145364684023	 -0.778120027566749
7  	 [-5.12        

In [29]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 18

np.random.seed(run_num_18)
surrogate_gp_18 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_18 = GPGO_multi(surrogate_gp_18, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_18.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [1.53983224 0.05584255]	 22.67190580753611	 22.67190580753611	    
init	 [ 3.87687906 -3.25795609]	 38.990099416711985	 22.67190580753611	    
init	 [3.60686662 2.56139557]	 56.66448698782129	 22.67190580753611	    
init	 [1.70088108 4.99604939]	 40.894059318256296	 22.67190580753611	    
init	 [-2.48864335 -4.83014733]	 54.6725749848372	 22.67190580753611	    
1  	 [-5.00718904  2.9174042 ]	 34.91005808676357	 22.67190580753611	 -0.797884560544953
2  	 [-5.12      -1.3722206]	 47.754213354155944	 22.67190580753611	 -0.7959246035321068
3  	 [-1.30205528  2.41793776]	 39.454406355016445	 22.67190580753611	 -0.7797500508825546
4  	 [ 1.04804663 -5.12      ]	 30.475340809730255	 22.67190580753611	 -0.7424631790896999
5  	 [-1.46191316 -1.33679079]	 38.826164708539096	 22.67190580753611	 -0.7344232114950985
6  	 [-2.73822073  5.12      ]	 47.16200446544369	 22.67190580753611	 -0.6563522776007924
7  	 [ 5

In [30]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 19

np.random.seed(run_num_19)
surrogate_gp_19 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_19 = GPGO_multi(surrogate_gp_19, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_19.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-4.12125592  2.6751971 ]	 41.43485596167127	 25.867003842388073	    
init	 [-2.59135515 -3.70553152]	 51.60126035043229	 25.867003842388073	    
init	 [-1.72598719 -4.27008445]	 43.97430127029199	 25.867003842388073	    
init	 [1.76104531 3.13952049]	 25.867003842388073	 25.867003842388073	    
init	 [4.9432772  1.38916592]	 44.66580306903559	 25.867003842388073	    
1  	 [ 5.12 -5.12]	 57.849427451571785	 25.867003842388073	 -0.7978845534957015
2  	 [ 1.77619034 -1.68663061]	 28.23864836494113	 25.867003842388073	 -0.7963184569461808
3  	 [5.12 5.12]	 57.849427451571785	 25.867003842388073	 -0.7843001945270276
4  	 [-1.28669736  5.12      ]	 42.86568994878547	 25.867003842388073	 -0.7798913632004989
5  	 [-5.12       -0.97841386]	 29.973843622719365	 25.867003842388073	 -0.7794667829832421
6	 [-1.20053952  0.35650344]	 24.713914812219098	 24.713914812219098	 -0.7733932280436672
7  	 [ 1.54014555 -5

In [31]:
### Bayesian optimisation runs (x20): 'GP' Acquisition Function run number = 20

np.random.seed(run_num_20)
surrogate_gp_20 = GaussianProcess(cov_func, optimize=hyperOpt)

gp_20 = GPGO_multi(surrogate_gp_20, Acquisition_new(util_gp), f_syn_polarity, param, n_jobs = -1) # define BayesOpt
gp_20.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [0.9024594  4.07258857]	 20.24255447774017	 17.388691338534382	    
init	 [4.00927467 3.23417577]	 35.55852254546393	 17.388691338534382	    
init	 [-4.75249064  1.96359764]	 36.54582989269194	 17.388691338534382	    
init	 [-1.24230715  0.18955208]	 17.388691338534382	 17.388691338534382	    
init	 [ 1.61742301 -3.13497377]	 33.22932870179905	 17.388691338534382	    
1  	 [-5.12       -3.80671405]	 49.927274972860054	 17.388691338534382	 -0.7978832538894429
2  	 [ 5.12       -0.92644509]	 30.83209553534477	 17.388691338534382	 -0.7954810507629072
3  	 [ 5.12 -5.12]	 57.849427451571785	 17.388691338534382	 -0.7942092645900973
4  	 [-2.70369185  5.12      ]	 49.103408966650434	 17.388691338534382	 -0.7858815232553523
5  	 [-1.50461024 -5.12      ]	 51.184370557302486	 17.388691338534382	 -0.7814906296366262
6  	 [1.91189043 0.46615148]	 25.140989672081915	 17.388691338534382	 -0.7167629908739096
7  	 

In [32]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 1 

np.random.seed(run_num_1)
surrogate_stp_1 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_1 = GPGO_multi_STP(surrogate_stp_1, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_1.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-0.84969467  2.25612281]	 20.33436270766351	 19.908403246996286	    
init	 [-5.1188288  -2.02411446]	 33.07414982069084	 19.908403246996286	    
init	 [-3.61721968 -4.17445279]	 53.347974723929894	 19.908403246996286	    
init	 [-3.21269544 -1.58145816]	 39.218472310354045	 19.908403246996286	    
init	 [-1.05710106  0.39748336]	 19.908403246996286	 19.908403246996286	    
1  	 [ 4.04077223 -4.24914728]	 44.655863836006326	 19.908403246996286	 -17.32897936958058
2  	 [3.88696178 5.12      ]	 46.45113190681991	 19.908403246996286	 -18.546885134830497
3  	 [-5.12  5.12]	 57.849427451571785	 19.908403246996286	 -19.569653346262545
4  	 [5.12       0.45084536]	 48.65481979333338	 19.908403246996286	 -21.29404527074605
5  	 [ 0.22552218 -5.12      ]	 37.44364330585873	 19.908403246996286	 -21.899666588998095
6  	 [ 1.74851334 -1.37741233]	 32.22540023390415	 19.908403246996286	 -21.75509927986315
7  	 [-

In [33]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 2 

np.random.seed(run_num_2)
surrogate_stp_2 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_2 = GPGO_multi_STP(surrogate_stp_2, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_2.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-0.6554122  -4.85451539]	 43.490296251903594	 18.964539418712707	    
init	 [ 0.50854377 -0.6622987 ]	 35.91861667536101	 18.964539418712707	    
init	 [-0.81543371 -1.73737143]	 20.479562046739524	 18.964539418712707	    
init	 [-3.02439799  1.2213347 ]	 18.964539418712707	 18.964539418712707	    
init	 [-2.05153614 -2.3876887 ]	 28.041315668371354	 18.964539418712707	    
1  	 [ 5.12 -5.12]	 57.849427451571785	 18.964539418712707	 -14.974779303390344
2  	 [3.56771682 5.12      ]	 60.761735270045946	 18.964539418712707	 -18.371932317001374
3  	 [5.12       0.26666643]	 40.041094463606925	 18.964539418712707	 -20.871230358641267
4  	 [-5.12  5.12]	 57.849427451571785	 18.964539418712707	 -21.008353730935795
5  	 [-5.12 -5.12]	 57.849427451571785	 18.964539418712707	 -22.335264933478456
6  	 [-0.77214363  5.12      ]	 38.13407864993035	 18.964539418712707	 -23.36519505009379
7  	 [-5.12       -1.2348

In [34]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 3 

np.random.seed(run_num_3)
surrogate_stp_3 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_3 = GPGO_multi_STP(surrogate_stp_3, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_3.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [0.52017052 2.1314337 ]	 27.953858411008774	 10.607662635789808	    
init	 [-2.14113547  0.11087468]	 10.607662635789808	 10.607662635789808	    
init	 [4.02377681 4.05804123]	 33.42749829480097	 10.607662635789808	    
init	 [-3.83400642 -2.99783293]	 28.650953928965198	 10.607662635789808	    
init	 [-4.59297584 -0.6061072 ]	 57.6631355589384	 10.607662635789808	    
1  	 [ 2.87042319 -1.98283743]	 15.364229918613852	 10.607662635789808	 -17.667277654047023
2  	 [-5.12  5.12]	 57.849427451571785	 10.607662635789808	 -16.545777429860973
3  	 [ 0.05045819 -5.12      ]	 29.42563030613281	 10.607662635789808	 -19.148409129229428
4  	 [ 5.12 -5.12]	 57.849427451571785	 10.607662635789808	 -18.84429460668159
5  	 [5.12       0.53388264]	 48.9839863181992	 10.607662635789808	 -20.50539795571924
6  	 [-1.23918423  5.12      ]	 39.78123915664758	 10.607662635789808	 -21.170833209212354
7  	 [-0.16488336 -1.

In [35]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 4 

np.random.seed(run_num_4)
surrogate_stp_4 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_4 = GPGO_multi_STP(surrogate_stp_4, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_4.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [4.78238555 0.48365823]	 51.03163809010808	 14.323038259018315	    
init	 [4.84028785 2.19971578]	 39.78645699016559	 14.323038259018315	    
init	 [ 2.02474316 -2.90724357]	 14.323038259018315	 14.323038259018315	    
init	 [ 4.87705042 -5.05620219]	 52.80627247106233	 14.323038259018315	    
init	 [-2.52946061 -0.66773471]	 41.61497868486559	 14.323038259018315	    
1  	 [-5.12        4.70342989]	 63.931476122515704	 14.323038259018315	 -20.75106290341067
2  	 [0.36418297 5.12      ]	 45.63186716478759	 14.323038259018315	 -23.37888720487815
3  	 [-5.12 -5.12]	 57.849427451571785	 14.323038259018315	 -23.60862070835514
4  	 [-0.95980985 -5.12      ]	 30.16309515482829	 14.323038259018315	 -24.660787713161547
5	 [0.85405947 1.14411555]	 9.783262796147048	 9.783262796147048	 -23.967582777885156
6  	 [-2.14784002  2.7179478 ]	 28.013666448413485	 9.783262796147048	 -22.832408011960904
7  	 [3.91753049

In [36]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 5 

np.random.seed(run_num_5)
surrogate_stp_5 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_5 = GPGO_multi_STP(surrogate_stp_5, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_5.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-2.84678993  3.79629882]	 33.93442008827236	 7.8108627039749745	    
init	 [-3.00319585  4.2865757 ]	 39.673876075575784	 7.8108627039749745	    
init	 [-0.11866943  1.14425716]	 7.8108627039749745	 7.8108627039749745	    
init	 [2.72289645 0.1886002 ]	 25.38160395721669	 7.8108627039749745	    
init	 [-2.08076286 -3.19773462]	 22.589982116319675	 7.8108627039749745	    
1  	 [ 5.12       -4.82496584]	 57.66701678941767	 7.8108627039749745	 -12.593288181171479
2  	 [5.12 5.12]	 57.849427451571785	 7.8108627039749745	 -16.743319693347694
3  	 [-5.12       -0.04632923]	 29.347558617657597	 7.8108627039749745	 -19.29827154561257
4  	 [1.0036767 5.12     ]	 29.934748896773826	 7.8108627039749745	 -18.971225560230806
5  	 [-5.12 -5.12]	 57.849427451571785	 7.8108627039749745	 -18.7355748824161
6  	 [ 1.10675511 -5.12      ]	 32.31613899686741	 7.8108627039749745	 -20.28367950208839
7  	 [ 5.12       -1.4

In [37]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 6 

np.random.seed(run_num_6)
surrogate_stp_6 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_6 = GPGO_multi_STP(surrogate_stp_6, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_6.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 4.02288795 -1.72052679]	 31.08835710146886	 17.28954482757088	    
init	 [ 3.28938622 -4.69302655]	 58.797867722203385	 17.28954482757088	    
init	 [-4.0175956   0.97333314]	 17.28954482757088	 17.28954482757088	    
init	 [ 0.30532979 -0.83141193]	 19.296253155889353	 17.28954482757088	    
init	 [-1.68542362  1.25459899]	 28.650630936276173	 17.28954482757088	    
1  	 [-3.50809801 -4.82870379]	 60.86418154947549	 17.28954482757088	 -17.235730660928617
2  	 [4.69607878 3.83657847]	 54.92048826283646	 17.28954482757088	 -20.383605617112707
3  	 [-5.12  5.12]	 57.849427451571785	 17.28954482757088	 -21.866426768136396
4  	 [0.23149004 5.12      ]	 37.81790604440799	 17.28954482757088	 -23.19503683131934
5  	 [-5.12       -1.81010865]	 38.513615318590574	 17.28954482757088	 -22.881641881752138
6  	 [-0.12005946 -4.2763893 ]	 32.665292710575464	 17.28954482757088	 -22.717453284908167
7	 [1.91645314 

In [38]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 7 

np.random.seed(run_num_7)
surrogate_stp_7 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_7 = GPGO_multi_STP(surrogate_stp_7, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_7.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-4.33860312  2.86636843]	 45.646133072936244	 22.52235437888213	    
init	 [-0.63068947  2.28828342]	 34.83012662845338	 22.52235437888213	    
init	 [4.8946126  0.39419771]	 44.09657005662876	 22.52235437888213	    
init	 [ 0.01147355 -4.38219639]	 36.613209822404315	 22.52235437888213	    
init	 [-2.37118484e+00 -1.20319155e-03]	 22.52235437888213	 22.52235437888213	    
1  	 [-5.12 -5.12]	 57.849427451571785	 22.52235437888213	 -18.934444602545486
2  	 [ 5.12 -5.12]	 57.849427451571785	 22.52235437888213	 -21.274757482549198
3  	 [3.59319651 5.12      ]	 60.16974170317036	 22.52235437888213	 -22.87332002187941
4  	 [-4.02558214 -1.33646422]	 33.289738493146906	 22.52235437888213	 -24.246292177738272
5	 [ 1.4059687  -0.96120265]	 21.501174989248653	 21.501174989248653	 -23.637035810048285
6  	 [-1.80943359  5.12      ]	 38.550630954849794	 21.501174989248653	 -22.72845967464331
7  	 [2.33861662 2.

In [39]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 8 

np.random.seed(run_num_8)
surrogate_stp_8 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_8 = GPGO_multi_STP(surrogate_stp_8, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_8.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [3.82391708 4.79785639]	 50.20079446939181	 13.871821018360485	    
init	 [3.78055209 0.31596228]	 36.5114251593508	 13.871821018360485	    
init	 [-2.73686192 -5.00327624]	 43.34985765011677	 13.871821018360485	    
init	 [-0.7119993  -0.99992207]	 13.871821018360485	 13.871821018360485	    
init	 [ 0.23218863 -0.22126801]	 17.190590355445654	 13.871821018360485	    
1  	 [-3.14011662  5.12      ]	 42.41645393736988	 13.871821018360485	 -17.626499924708103
2  	 [ 5.12 -5.12]	 57.849427451571785	 13.871821018360485	 -18.55893447809943
3  	 [-5.12        0.36878666]	 45.85041615780568	 13.871821018360485	 -20.7009735440955
4  	 [ 1.18398521 -4.4098634 ]	 45.25747845830972	 13.871821018360485	 -21.247716270295587
5  	 [0.23149879 3.35079831]	 36.03995068376803	 13.871821018360485	 -21.633727767364014
6  	 [-5.12      -2.9446905]	 38.19371317602073	 13.871821018360485	 -21.373312264550222
7  	 [-2.35498

In [40]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 9 

np.random.seed(run_num_9)
surrogate_stp_9 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_9 = GPGO_multi_STP(surrogate_stp_9, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_9.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-5.01376866  0.01919582]	 25.248289026162446	 24.454800313488693	    
init	 [-0.04328148 -3.74958562]	 24.454800313488693	 24.454800313488693	    
init	 [-3.66478248 -2.88195916]	 39.46509426509438	 24.454800313488693	    
init	 [-0.83447623 -2.57944404]	 31.06766397812992	 24.454800313488693	    
init	 [-4.25922917 -1.58209393]	 49.922543556206975	 24.454800313488693	    
1  	 [3.11029524 4.9495987 ]	 36.97642606974743	 24.454800313488693	 -16.220192105908154
2  	 [ 5.12       -1.53068924]	 51.08238892053614	 24.454800313488693	 -16.878403904249524
3  	 [-2.66822357  5.12      ]	 50.9591753611455	 24.454800313488693	 -18.665908273836425
4  	 [0.30427927 1.55525954]	 35.25942660827667	 24.454800313488693	 -19.933565980313357
5  	 [ 3.58500187 -5.12      ]	 60.38431260813631	 24.454800313488693	 -19.877021436064812
6  	 [5.12       2.10915119]	 35.63421445174545	 24.454800313488693	 -21.4135006003207

In [41]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 10 

np.random.seed(run_num_10)
surrogate_stp_10 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_10 = GPGO_multi_STP(surrogate_stp_10, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_10.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 2.77832339 -4.90750004]	 41.674330194390116	 13.839458310244165	    
init	 [1.36855793 2.54775176]	 44.69639719065837	 13.839458310244165	    
init	 [-0.01528819 -2.81808235]	 13.839458310244165	 13.839458310244165	    
init	 [-3.09183626  2.66783449]	 33.23221510904937	 13.839458310244165	    
init	 [-3.38830503 -4.2154003 ]	 54.73014366983691	 13.839458310244165	    
1  	 [ 5.12       -0.56895878]	 48.32435982022352	 13.839458310244165	 -20.166092853867085
2  	 [5.12 5.12]	 57.849427451571785	 13.839458310244165	 -21.212442074225052
3  	 [-5.12       -0.60622128]	 47.14650429686387	 13.839458310244165	 -22.813395438580955
4  	 [-5.12  5.12]	 57.849427451571785	 13.839458310244165	 -23.142687897080016
5  	 [-0.83770616  5.12      ]	 34.39043972230266	 13.839458310244165	 -24.129087758019484
6	 [-1.2319285  -0.10213748]	 12.38455013497624	 12.38455013497624	 -23.6746839220942
7  	 [ 1.89900326 -0.7

In [42]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 11 

np.random.seed(run_num_11)
surrogate_stp_11 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_11 = GPGO_multi_STP(surrogate_stp_11, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_11.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-3.27403839 -4.92057353]	 47.655641290890955	 10.679755252484755	    
init	 [-0.37664229  2.30332343]	 35.87889240695259	 10.679755252484755	    
init	 [-0.81711509 -0.14922651]	 10.679755252484755	 10.679755252484755	    
init	 [-4.98912446 -0.12931474]	 28.05462905574235	 10.679755252484755	    
init	 [4.52410012 3.59214172]	 71.62694632141611	 10.679755252484755	    
1  	 [ 3.03550212 -4.46785095]	 49.22040431745476	 10.679755252484755	 -22.027122748758053
2  	 [-5.12  5.12]	 57.849427451571785	 10.679755252484755	 -22.827172378296837
3  	 [ 5.12       -0.75126026]	 39.40992212508443	 10.679755252484755	 -24.144839397179346
4  	 [1.21059641 5.12      ]	 37.93967203511711	 10.679755252484755	 -23.819501208262327
5  	 [-0.21477288 -3.12951005]	 20.77672407531348	 10.679755252484755	 -23.530764882147995
6  	 [ 1.98006025 -0.29558095]	 16.91133137957369	 10.679755252484755	 -22.6222314941886
7  	 [-1

In [43]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 12

np.random.seed(run_num_12)
surrogate_stp_12 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_12 = GPGO_multi_STP(surrogate_stp_12, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_12.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-3.54137249  2.45810889]	 57.903242869085595	 40.51116653209555	    
init	 [-2.42365424  0.34549139]	 40.51116653209555	 40.51116653209555	    
init	 [-4.97075238  4.28796936]	 55.62655915398208	 40.51116653209555	    
init	 [ 4.10332011 -4.77776458]	 49.962803461970296	 40.51116653209555	    
init	 [ 4.6791612  -3.71497655]	 62.183891474990624	 40.51116653209555	    
1  	 [5.12       3.15380098]	 43.18819096116633	 40.51116653209555	 -25.58536380060787
2  	 [-5.12       -4.55821268]	 69.0405348730723	 40.51116653209555	 -25.339476101184164
3  	 [0.62705866 5.12      ]	 46.2969310725815	 40.51116653209555	 -27.256734303791088
4  	 [-0.45339239 -5.12      ]	 48.70454519365059	 40.51116653209555	 -27.069321669425
5	 [ 1.96981416 -0.06291409]	 4.834640795711783	 4.834640795711783	 -27.046672806954035
6  	 [ 0.18864167 -1.87357606]	 12.777887442066387	 4.834640795711783	 -25.670455599904894
7  	 [ 5.12 

In [44]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 13 

np.random.seed(run_num_13)
surrogate_stp_13 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_13 = GPGO_multi_STP(surrogate_stp_13, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_13.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 2.84367268 -2.68757791]	 33.58019830007169	 17.435826639425656	    
init	 [3.32061217 4.76927179]	 56.857057997503354	 17.435826639425656	    
init	 [ 4.83943541 -0.47667971]	 48.211919361679726	 17.435826639425656	    
init	 [1.11659482 2.82139151]	 17.435826639425656	 17.435826639425656	    
init	 [1.45012065 2.27346667]	 38.25352329493884	 17.435826639425656	    
1  	 [-4.10876704 -2.60547476]	 43.79941556238635	 17.435826639425656	 -20.28104909624227
2  	 [-4.86899536  5.12      ]	 55.832509686098035	 17.435826639425656	 -20.877349268264844
3  	 [-0.62139712 -5.12      ]	 46.54016237374039	 17.435826639425656	 -22.354588596967545
4  	 [-2.71929057  1.3816919 ]	 38.58323714182029	 17.435826639425656	 -22.74311201660965
5  	 [-0.99650648  5.12      ]	 29.920147900297124	 17.435826639425656	 -22.58059118056724
6  	 [ 5.12 -5.12]	 57.849427451571785	 17.435826639425656	 -22.08368319958781
7  	 [-0.

In [45]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 14 

np.random.seed(run_num_14)
surrogate_stp_14 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_14 = GPGO_multi_STP(surrogate_stp_14, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_14.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [0.14277984 2.79721013]	 18.683085263052178	 10.423838604848608	    
init	 [ 3.7931795  -5.03759925]	 47.36348784446708	 10.423838604848608	    
init	 [-1.94830412  4.68586229]	 40.19779334078636	 10.423838604848608	    
init	 [ 0.13431513 -1.86076749]	 10.423838604848608	 10.423838604848608	    
init	 [ 0.40140736 -2.85434939]	 30.353548735049138	 10.423838604848608	    
1  	 [-4.71265632 -5.12      ]	 63.45874543051727	 10.423838604848608	 -16.107786530539233
2  	 [5.12 5.12]	 57.849427451571785	 10.423838604848608	 -19.931416277165436
3  	 [-5.12        0.35628846]	 45.24449218978817	 10.423838604848608	 -21.78892286294519
4  	 [ 5.12      -0.0638437]	 29.722632282670638	 10.423838604848608	 -22.16171693467924
5  	 [-5.12  5.12]	 57.849427451571785	 10.423838604848608	 -21.593973389041132
6  	 [-1.66669145  0.07860839]	 18.97783633316541	 10.423838604848608	 -22.69007628137382
7  	 [1.92611222 0.2

In [46]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 15 

np.random.seed(run_num_15)
surrogate_stp_15 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_15 = GPGO_multi_STP(surrogate_stp_15, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_15.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [ 3.57189322 -3.28810573]	 54.938487770767075	 7.990765314336182	    
init	 [-4.56332069 -1.41784631]	 60.750198753157726	 7.990765314336182	    
init	 [-2.29989449  0.3072023 ]	 31.984997246800887	 7.990765314336182	    
init	 [-1.9873903  -2.00218256]	 7.990765314336182	 7.990765314336182	    
init	 [-3.97576933 -2.5610341 ]	 41.754957769694336	 7.990765314336182	    
1  	 [4.02234348 3.94993692]	 32.370291610486674	 7.990765314336182	 -20.838724785528175
2  	 [-5.12        5.07072939]	 55.608347879327546	 7.990765314336182	 -20.43812295496221
3  	 [-0.40553392  5.12      ]	 47.37878697281484	 7.990765314336182	 -21.976706372257205
4  	 [-0.33066276 -5.12      ]	 43.888038253833685	 7.990765314336182	 -22.471523092050024
5  	 [1.78106216 0.55456205]	 30.95848813293346	 7.990765314336182	 -22.647220175302547
6  	 [5.12       0.26624152]	 40.0143126056433	 7.990765314336182	 -22.162230560835063
7  	 

In [47]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 16 

np.random.seed(run_num_16)
surrogate_stp_16 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_16 = GPGO_multi_STP(surrogate_stp_16, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_16.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-2.83349935  0.23719262]	 22.27210456874814	 22.27210456874814	    
init	 [ 0.51918292 -4.65303603]	 57.57021076017139	 22.27210456874814	    
init	 [-1.42613673 -2.83565116]	 33.89145899403749	 22.27210456874814	    
init	 [ 1.9325559  -3.44339021]	 35.85029586225333	 22.27210456874814	    
init	 [-4.39987336  4.51595121]	 77.78800881964571	 22.27210456874814	    
1  	 [2.57149196 4.50237179]	 65.89077218553832	 22.27210456874814	 -24.562114042346174
2  	 [5.12       0.18193806]	 34.81051778742929	 22.27210456874814	 -26.563736870410843
3  	 [-5.12 -5.12]	 57.849427451571785	 22.27210456874814	 -25.757345547621124
4	 [1.04281874 0.62517362]	 18.901392725485252	 18.901392725485252	 -26.50006864518259
5  	 [ 5.12 -5.12]	 57.849427451571785	 18.901392725485252	 -25.351918246096893
6  	 [-0.91788827  3.46846713]	 33.978776658751684	 18.901392725485252	 -25.95902375725217
7  	 [-5.12       -1.65660824]	

In [48]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 17 

np.random.seed(run_num_17)
surrogate_stp_17 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_17 = GPGO_multi_STP(surrogate_stp_17, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_17.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-2.10263037  0.31320838]	 20.395145364684023	 20.395145364684023	    
init	 [-3.15882714 -4.42470033]	 53.03732051200137	 20.395145364684023	    
init	 [2.93873111 1.60085526]	 29.989224812583537	 20.395145364684023	    
init	 [1.40821398 0.77417363]	 29.451989415882437	 20.395145364684023	    
init	 [-4.71999574 -1.45598869]	 55.89242173757483	 20.395145364684023	    
1  	 [ 4.96008736 -4.44720822]	 64.14783177084814	 20.395145364684023	 -20.04218100529065
2  	 [-5.12        4.84467889]	 56.79149209490958	 20.395145364684023	 -22.868964017279033
3  	 [-0.25166648  5.12      ]	 39.09275594104926	 20.395145364684023	 -24.086219926605118
4  	 [5.12 5.12]	 57.849427451571785	 20.395145364684023	 -23.81702285288551
5  	 [ 0.87311321 -3.37293667]	 32.13108852329271	 20.395145364684023	 -24.739093327259248
6  	 [ 5.12       -0.83011426]	 34.78997678324947	 20.395145364684023	 -24.132268690207944
7  	 [-5.

In [49]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 18 

np.random.seed(run_num_18)
surrogate_stp_18 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_18 = GPGO_multi_STP(surrogate_stp_18, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_18.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [1.53983224 0.05584255]	 22.67190580753611	 22.67190580753611	    
init	 [ 3.87687906 -3.25795609]	 38.990099416711985	 22.67190580753611	    
init	 [3.60686662 2.56139557]	 56.66448698782129	 22.67190580753611	    
init	 [1.70088108 4.99604939]	 40.894059318256296	 22.67190580753611	    
init	 [-2.48864335 -4.83014733]	 54.6725749848372	 22.67190580753611	    
1  	 [-5.12        2.93383941]	 38.3837857926325	 22.67190580753611	 -22.377416780752057
2  	 [-5.12       -1.37707184]	 47.98355623109579	 22.67190580753611	 -22.205882704211653
3  	 [-1.35215233  2.41768156]	 42.352180573195184	 22.67190580753611	 -22.76598310231805
4  	 [ 1.04785641 -5.12      ]	 30.47139536039345	 22.67190580753611	 -22.786922460075758
5  	 [-1.45596366 -1.34767085]	 39.31453219916528	 22.67190580753611	 -22.242436179486997
6  	 [ 5.12       -0.35123804]	 44.98868812677667	 22.67190580753611	 -22.10638376785207
7  	 [-2.80

In [50]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 19 

np.random.seed(run_num_19)
surrogate_stp_19 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_19 = GPGO_multi_STP(surrogate_stp_19, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_19.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [-4.12125592  2.6751971 ]	 41.43485596167127	 25.867003842388073	    
init	 [-2.59135515 -3.70553152]	 51.60126035043229	 25.867003842388073	    
init	 [-1.72598719 -4.27008445]	 43.97430127029199	 25.867003842388073	    
init	 [1.76104531 3.13952049]	 25.867003842388073	 25.867003842388073	    
init	 [4.9432772  1.38916592]	 44.66580306903559	 25.867003842388073	    
1  	 [ 4.73730587 -5.12      ]	 62.16353074996998	 25.867003842388073	 -20.076752102273044
2  	 [ 1.58908083 -1.47144503]	 43.00389274319619	 25.867003842388073	 -22.65425751639521
3  	 [5.12 5.12]	 57.849427451571785	 25.867003842388073	 -22.77642255156076
4  	 [-1.28683221  5.12      ]	 42.87428498664837	 25.867003842388073	 -23.952627385630123
5  	 [-5.12      -0.9783881]	 29.974012146498225	 25.867003842388073	 -23.91714385989046
6  	 [-1.40429182  0.40754384]	 38.74342644939308	 25.867003842388073	 -23.312185036808614
7  	 [ 1.3323

In [51]:
### Bayesian optimisation runs (x20): 'STP' Acquisition Function run number = 20 

np.random.seed(run_num_20)
surrogate_stp_20 = tStudentProcess(cov_func, nu = df, optimize=hyperOpt)

stp_20 = GPGO_multi_STP(surrogate_stp_20, Acquisition_new(util_stp), f_syn_polarity, param, n_jobs = -1) # Define BayesOpt
stp_20.run(max_iter = max_iter, init_evals = n_init) # run


Evaluation 	 Proposed point 	  Current eval. 	  Best eval. 	 Min. ApproxAcqFunc 
init	 [0.9024594  4.07258857]	 20.24255447774017	 17.388691338534382	    
init	 [4.00927467 3.23417577]	 35.55852254546393	 17.388691338534382	    
init	 [-4.75249064  1.96359764]	 36.54582989269194	 17.388691338534382	    
init	 [-1.24230715  0.18955208]	 17.388691338534382	 17.388691338534382	    
init	 [ 1.61742301 -3.13497377]	 33.22932870179905	 17.388691338534382	    
1  	 [-4.87904838 -3.98922798]	 42.49332305798536	 17.388691338534382	 -14.957074443856806
2  	 [ 5.12       -0.92643146]	 30.83245214836478	 17.388691338534382	 -16.434253028691735
3  	 [ 5.12 -5.12]	 57.849427451571785	 17.388691338534382	 -16.503954169691625
4  	 [-2.7037672  5.12     ]	 49.0992806463018	 17.388691338534382	 -18.83524424726938
5  	 [-1.36264737 -5.12      ]	 47.2830402664146	 17.388691338534382	 -19.7965914869068
6  	 [1.91306258 0.46597355]	 25.104450841908065	 17.388691338534382	 -20.3814824699662
7  	 [-3.93918311

In [52]:
### Simple regret minimisation: run number = 1

gp_output_1 = np.append(np.max(gp_1.GP.y[0:n_init]),gp_1.GP.y[n_init:(n_init+max_iter)]) 
stp_output_1 = np.append(np.max(stp_1.GP.y[0:n_init]),stp_1.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_1 = np.log(y_global_orig - gp_output_1)
regret_stp_1 = np.log(y_global_orig - stp_output_1)

simple_regret_gp_1 = min_max_array(regret_gp_1)
simple_regret_stp_1 = min_max_array(regret_stp_1)

min_simple_regret_gp_1 = min(simple_regret_gp_1)
min_simple_regret_stp_1 = min(simple_regret_stp_1)

min_simple_regret_gp_1, min_simple_regret_stp_1


(2.9911419163163977, 2.7664117731206583)

In [53]:
### Simple regret minimisation: run number = 2

gp_output_2 = np.append(np.max(gp_2.GP.y[0:n_init]),gp_2.GP.y[n_init:(n_init+max_iter)]) 
stp_output_2 = np.append(np.max(stp_2.GP.y[0:n_init]),stp_2.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_2 = np.log(y_global_orig - gp_output_2)
regret_stp_2 = np.log(y_global_orig - stp_output_2)

simple_regret_gp_2 = min_max_array(regret_gp_2)
simple_regret_stp_2 = min_max_array(regret_stp_2)

min_simple_regret_gp_2 = min(simple_regret_gp_2)
min_simple_regret_stp_2 = min(simple_regret_stp_2)

min_simple_regret_gp_2, min_simple_regret_stp_2


(2.70208273321717, 2.7859179997878267)

In [54]:
### Simple regret minimisation: run number = 3

gp_output_3 = np.append(np.max(gp_3.GP.y[0:n_init]),gp_3.GP.y[n_init:(n_init+max_iter)]) 
stp_output_3 = np.append(np.max(stp_3.GP.y[0:n_init]),stp_3.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_3 = np.log(y_global_orig - gp_output_3)
regret_stp_3 = np.log(y_global_orig - stp_output_3)

simple_regret_gp_3 = min_max_array(regret_gp_3)
simple_regret_stp_3 = min_max_array(regret_stp_3)

min_simple_regret_gp_3 = min(simple_regret_gp_3)
min_simple_regret_stp_3 = min(simple_regret_stp_3)

min_simple_regret_gp_3, min_simple_regret_stp_3


(2.361576630127625, 2.361576630127625)

In [55]:
### Simple regret minimisation: run number = 4

gp_output_4 = np.append(np.max(gp_4.GP.y[0:n_init]),gp_4.GP.y[n_init:(n_init+max_iter)]) 
stp_output_4 = np.append(np.max(stp_4.GP.y[0:n_init]),stp_4.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_4 = np.log(y_global_orig - gp_output_4)
regret_stp_4 = np.log(y_global_orig - stp_output_4)

simple_regret_gp_4 = min_max_array(regret_gp_4)
simple_regret_stp_4 = min_max_array(regret_stp_4)

min_simple_regret_gp_4 = min(simple_regret_gp_4)
min_simple_regret_stp_4 = min(simple_regret_stp_4)

min_simple_regret_gp_4, min_simple_regret_stp_4


(2.3258106988104013, 2.280673047646142)

In [56]:
### Simple regret minimisation: run number = 5

gp_output_5 = np.append(np.max(gp_5.GP.y[0:n_init]),gp_5.GP.y[n_init:(n_init+max_iter)]) 
stp_output_5 = np.append(np.max(stp_5.GP.y[0:n_init]),stp_5.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_5 = np.log(y_global_orig - gp_output_5)
regret_stp_5 = np.log(y_global_orig - stp_output_5)

simple_regret_gp_5 = min_max_array(regret_gp_5)
simple_regret_stp_5 = min_max_array(regret_stp_5)

min_simple_regret_gp_5 = min(simple_regret_gp_5)
min_simple_regret_stp_5 = min(simple_regret_stp_5)

min_simple_regret_gp_5, min_simple_regret_stp_5


(1.8240861705129932, 2.05551541920764)

In [57]:
### Simple regret minimisation: run number = 6

gp_output_6 = np.append(np.max(gp_6.GP.y[0:n_init]),gp_6.GP.y[n_init:(n_init+max_iter)]) 
stp_output_6 = np.append(np.max(stp_6.GP.y[0:n_init]),stp_6.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_6 = np.log(y_global_orig - gp_output_6)
regret_stp_6 = np.log(y_global_orig - stp_output_6)

simple_regret_gp_6 = min_max_array(regret_gp_6)
simple_regret_stp_6 = min_max_array(regret_stp_6)

min_simple_regret_gp_6 = min(simple_regret_gp_6)
min_simple_regret_stp_6 = min(simple_regret_stp_6)

min_simple_regret_gp_6, min_simple_regret_stp_6


(2.8190077613254987, 2.5999421618511445)

In [58]:
### Simple regret minimisation: run number = 7

gp_output_7 = np.append(np.max(gp_7.GP.y[0:n_init]),gp_7.GP.y[n_init:(n_init+max_iter)]) 
stp_output_7 = np.append(np.max(stp_7.GP.y[0:n_init]),stp_7.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_7 = np.log(y_global_orig - gp_output_7)
regret_stp_7 = np.log(y_global_orig - stp_output_7)

simple_regret_gp_7 = min_max_array(regret_gp_7)
simple_regret_stp_7 = min_max_array(regret_stp_7)

min_simple_regret_gp_7 = min(simple_regret_gp_7)
min_simple_regret_stp_7 = min(simple_regret_stp_7)

min_simple_regret_gp_7, min_simple_regret_stp_7


(2.5470487663770065, 2.741911564707682)

In [59]:
### Simple regret minimisation: run number = 8

gp_output_8 = np.append(np.max(gp_8.GP.y[0:n_init]),gp_8.GP.y[n_init:(n_init+max_iter)]) 
stp_output_8 = np.append(np.max(stp_8.GP.y[0:n_init]),stp_8.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_8 = np.log(y_global_orig - gp_output_8)
regret_stp_8 = np.log(y_global_orig - stp_output_8)

simple_regret_gp_8 = min_max_array(regret_gp_8)
simple_regret_stp_8 = min_max_array(regret_stp_8)

min_simple_regret_gp_8 = min(simple_regret_gp_8)
min_simple_regret_stp_8 = min(simple_regret_stp_8)

min_simple_regret_gp_8, min_simple_regret_stp_8


(2.6298595175876263, 2.6298595175876263)

In [60]:
### Simple regret minimisation: run number = 9

gp_output_9 = np.append(np.max(gp_9.GP.y[0:n_init]),gp_9.GP.y[n_init:(n_init+max_iter)]) 
stp_output_9 = np.append(np.max(stp_9.GP.y[0:n_init]),stp_9.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_9 = np.log(y_global_orig - gp_output_9)
regret_stp_9 = np.log(y_global_orig - stp_output_9)

simple_regret_gp_9 = min_max_array(regret_gp_9)
simple_regret_stp_9 = min_max_array(regret_stp_9)

min_simple_regret_gp_9 = min(simple_regret_gp_9)
min_simple_regret_stp_9 = min(simple_regret_stp_9)

min_simple_regret_gp_9, min_simple_regret_stp_9


(2.7482040141063537, 2.1416859222928637)

In [61]:
### Simple regret minimisation: run number = 10

gp_output_10 = np.append(np.max(gp_10.GP.y[0:n_init]),gp_10.GP.y[n_init:(n_init+max_iter)]) 
stp_output_10 = np.append(np.max(stp_10.GP.y[0:n_init]),stp_10.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_10 = np.log(y_global_orig - gp_output_10)
regret_stp_10 = np.log(y_global_orig - stp_output_10)

simple_regret_gp_10 = min_max_array(regret_gp_10)
simple_regret_stp_10 = min_max_array(regret_stp_10)

min_simple_regret_gp_10 = min(simple_regret_gp_10)
min_simple_regret_stp_10 = min(simple_regret_stp_10)

min_simple_regret_gp_10, min_simple_regret_stp_10


(2.523906518679678, 2.5084183153536896)

In [62]:
### Simple regret minimisation: run number = 11

gp_output_11 = np.append(np.max(gp_11.GP.y[0:n_init]),gp_11.GP.y[n_init:(n_init+max_iter)]) 
stp_output_11 = np.append(np.max(stp_11.GP.y[0:n_init]),stp_11.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_11 = np.log(y_global_orig - gp_output_11)
regret_stp_11 = np.log(y_global_orig - stp_output_11)

simple_regret_gp_11 = min_max_array(regret_gp_11)
simple_regret_stp_11 = min_max_array(regret_stp_11)

min_simple_regret_gp_11 = min(simple_regret_gp_11)
min_simple_regret_stp_11 = min(simple_regret_stp_11)

min_simple_regret_gp_11, min_simple_regret_stp_11


(2.3683499168354514, 2.3683499168354514)

In [63]:
### Simple regret minimisation: run number = 12

gp_output_12 = np.append(np.max(gp_12.GP.y[0:n_init]),gp_12.GP.y[n_init:(n_init+max_iter)]) 
stp_output_12 = np.append(np.max(stp_12.GP.y[0:n_init]),stp_12.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_12 = np.log(y_global_orig - gp_output_12)
regret_stp_12 = np.log(y_global_orig - stp_output_12)

simple_regret_gp_12 = min_max_array(regret_gp_12)
simple_regret_stp_12 = min_max_array(regret_stp_12)

min_simple_regret_gp_12 = min(simple_regret_gp_12)
min_simple_regret_stp_12 = min(simple_regret_stp_12)

min_simple_regret_gp_12, min_simple_regret_stp_12


(1.0136539055177785, 1.5664555746326727)

In [64]:
### Simple regret minimisation: run number = 13

gp_output_13 = np.append(np.max(gp_13.GP.y[0:n_init]),gp_13.GP.y[n_init:(n_init+max_iter)]) 
stp_output_13 = np.append(np.max(stp_13.GP.y[0:n_init]),stp_13.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_13 = np.log(y_global_orig - gp_output_13)
regret_stp_13 = np.log(y_global_orig - stp_output_13)

simple_regret_gp_13 = min_max_array(regret_gp_13)
simple_regret_stp_13 = min_max_array(regret_stp_13)

min_simple_regret_gp_13 = min(simple_regret_gp_13)
min_simple_regret_stp_13 = min(simple_regret_stp_13)

min_simple_regret_gp_13, min_simple_regret_stp_13


(2.858527091646381, 2.348066091343282)

In [65]:
### Simple regret minimisation: run number = 14

gp_output_14 = np.append(np.max(gp_14.GP.y[0:n_init]),gp_14.GP.y[n_init:(n_init+max_iter)]) 
stp_output_14 = np.append(np.max(stp_14.GP.y[0:n_init]),stp_14.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_14 = np.log(y_global_orig - gp_output_14)
regret_stp_14 = np.log(y_global_orig - stp_output_14)

simple_regret_gp_14 = min_max_array(regret_gp_14)
simple_regret_stp_14 = min_max_array(regret_stp_14)

min_simple_regret_gp_14 = min(simple_regret_gp_14)
min_simple_regret_stp_14 = min(simple_regret_stp_14)

min_simple_regret_gp_14, min_simple_regret_stp_14


(2.3440953566682032, 2.3440953566682032)

In [66]:
### Simple regret minimisation: run number = 15

gp_output_15 = np.append(np.max(gp_15.GP.y[0:n_init]),gp_15.GP.y[n_init:(n_init+max_iter)]) 
stp_output_15 = np.append(np.max(stp_15.GP.y[0:n_init]),stp_15.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_15 = np.log(y_global_orig - gp_output_15)
regret_stp_15 = np.log(y_global_orig - stp_output_15)

simple_regret_gp_15 = min_max_array(regret_gp_15)
simple_regret_stp_15 = min_max_array(regret_stp_15)

min_simple_regret_gp_15 = min(simple_regret_gp_15)
min_simple_regret_stp_15 = min(simple_regret_stp_15)

min_simple_regret_gp_15, min_simple_regret_stp_15


(2.078286539213237, 2.078286539213237)

In [67]:
### Simple regret minimisation: run number = 16

gp_output_16 = np.append(np.max(gp_16.GP.y[0:n_init]),gp_16.GP.y[n_init:(n_init+max_iter)]) 
stp_output_16 = np.append(np.max(stp_16.GP.y[0:n_init]),stp_16.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_16 = np.log(y_global_orig - gp_output_16)
regret_stp_16 = np.log(y_global_orig - stp_output_16)

simple_regret_gp_16 = min_max_array(regret_gp_16)
simple_regret_stp_16 = min_max_array(regret_stp_16)

min_simple_regret_gp_16 = min(simple_regret_gp_16)
min_simple_regret_stp_16 = min(simple_regret_stp_16)

min_simple_regret_gp_16, min_simple_regret_stp_16


(1.887677297341973, 1.5299692660229272)

In [68]:
### Simple regret minimisation: run number = 17

gp_output_17 = np.append(np.max(gp_17.GP.y[0:n_init]),gp_17.GP.y[n_init:(n_init+max_iter)]) 
stp_output_17 = np.append(np.max(stp_17.GP.y[0:n_init]),stp_17.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_17 = np.log(y_global_orig - gp_output_17)
regret_stp_17 = np.log(y_global_orig - stp_output_17)

simple_regret_gp_17 = min_max_array(regret_gp_17)
simple_regret_stp_17 = min_max_array(regret_stp_17)

min_simple_regret_gp_17 = min(simple_regret_gp_17)
min_simple_regret_stp_17 = min(simple_regret_stp_17)

min_simple_regret_gp_17, min_simple_regret_stp_17


(2.4422620047633656, 2.5127605610853845)

In [69]:
### Simple regret minimisation: run number = 18

gp_output_18 = np.append(np.max(gp_18.GP.y[0:n_init]),gp_18.GP.y[n_init:(n_init+max_iter)]) 
stp_output_18 = np.append(np.max(stp_18.GP.y[0:n_init]),stp_18.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_18 = np.log(y_global_orig - gp_output_18)
regret_stp_18 = np.log(y_global_orig - stp_output_18)

simple_regret_gp_18 = min_max_array(regret_gp_18)
simple_regret_stp_18 = min_max_array(regret_stp_18)

min_simple_regret_gp_18 = min(simple_regret_gp_18)
min_simple_regret_stp_18 = min(simple_regret_stp_18)

min_simple_regret_gp_18, min_simple_regret_stp_18


(2.4258079232777003, 2.566878139868196)

In [70]:
### Simple regret minimisation: run number = 19

gp_output_19 = np.append(np.max(gp_19.GP.y[0:n_init]),gp_19.GP.y[n_init:(n_init+max_iter)]) 
stp_output_19 = np.append(np.max(stp_19.GP.y[0:n_init]),stp_19.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_19 = np.log(y_global_orig - gp_output_19)
regret_stp_19 = np.log(y_global_orig - stp_output_19)

simple_regret_gp_19 = min_max_array(regret_gp_19)
simple_regret_stp_19 = min_max_array(regret_stp_19)

min_simple_regret_gp_19 = min(simple_regret_gp_19)
min_simple_regret_stp_19 = min(simple_regret_stp_19)

min_simple_regret_gp_19, min_simple_regret_stp_19


(2.9653388987769884, 2.0759645646647065)

In [71]:
### Simple regret minimisation: run number = 20

gp_output_20 = np.append(np.max(gp_20.GP.y[0:n_init]),gp_20.GP.y[n_init:(n_init+max_iter)]) 
stp_output_20 = np.append(np.max(stp_20.GP.y[0:n_init]),stp_20.GP.y[n_init:(n_init+max_iter)]) 

regret_gp_20 = np.log(y_global_orig - gp_output_20)
regret_stp_20 = np.log(y_global_orig - stp_output_20)

simple_regret_gp_20 = min_max_array(regret_gp_20)
simple_regret_stp_20 = min_max_array(regret_stp_20)

min_simple_regret_gp_20 = min(simple_regret_gp_20)
min_simple_regret_stp_20 = min(simple_regret_stp_20)

min_simple_regret_gp_20, min_simple_regret_stp_20


(2.8558200718561926, 2.8558200718561926)

In [72]:
# Iteration1 :

slice1 = 0

gp1 = [simple_regret_gp_1[slice1],
       simple_regret_gp_2[slice1],
       simple_regret_gp_3[slice1],
       simple_regret_gp_4[slice1],
       simple_regret_gp_5[slice1],
       simple_regret_gp_6[slice1],
       simple_regret_gp_7[slice1],
       simple_regret_gp_8[slice1],
       simple_regret_gp_9[slice1],
       simple_regret_gp_10[slice1],
       simple_regret_gp_11[slice1],
       simple_regret_gp_12[slice1],
       simple_regret_gp_13[slice1],
       simple_regret_gp_14[slice1],
       simple_regret_gp_15[slice1],
       simple_regret_gp_16[slice1],
       simple_regret_gp_17[slice1],
       simple_regret_gp_18[slice1],
       simple_regret_gp_19[slice1],
       simple_regret_gp_20[slice1]]

stp1 = [simple_regret_stp_1[slice1],
       simple_regret_stp_2[slice1],
       simple_regret_stp_3[slice1],
       simple_regret_stp_4[slice1],
       simple_regret_stp_5[slice1],
       simple_regret_stp_6[slice1],
       simple_regret_stp_7[slice1],
       simple_regret_stp_8[slice1],
       simple_regret_stp_9[slice1],
       simple_regret_stp_10[slice1],
       simple_regret_stp_11[slice1],
       simple_regret_stp_12[slice1],
       simple_regret_stp_13[slice1],
       simple_regret_stp_14[slice1],
       simple_regret_stp_15[slice1],
       simple_regret_stp_16[slice1],
       simple_regret_stp_17[slice1],
       simple_regret_stp_18[slice1],
       simple_regret_stp_19[slice1],
       simple_regret_stp_20[slice1]]

gp1_results = pd.DataFrame(gp1).sort_values(by=[0], ascending=False)
stp1_results = pd.DataFrame(stp1).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp1 = np.asarray(gp1_results[4:5][0])[0]
median_gp1 = np.asarray(gp1_results[9:10][0])[0]
upper_gp1 = np.asarray(gp1_results[14:15][0])[0]

lower_stp1 = np.asarray(stp1_results[4:5][0])[0]
median_stp1 = np.asarray(stp1_results[9:10][0])[0]
upper_stp1 = np.asarray(stp1_results[14:15][0])[0]


In [73]:
# Iteration11 :

slice11 = 10

gp11 = [simple_regret_gp_1[slice11],
       simple_regret_gp_2[slice11],
       simple_regret_gp_3[slice11],
       simple_regret_gp_4[slice11],
       simple_regret_gp_5[slice11],
       simple_regret_gp_6[slice11],
       simple_regret_gp_7[slice11],
       simple_regret_gp_8[slice11],
       simple_regret_gp_9[slice11],
       simple_regret_gp_10[slice11],
       simple_regret_gp_11[slice11],
       simple_regret_gp_12[slice11],
       simple_regret_gp_13[slice11],
       simple_regret_gp_14[slice11],
       simple_regret_gp_15[slice11],
       simple_regret_gp_16[slice11],
       simple_regret_gp_17[slice11],
       simple_regret_gp_18[slice11],
       simple_regret_gp_19[slice11],
       simple_regret_gp_20[slice11]]

stp11 = [simple_regret_stp_1[slice11],
       simple_regret_stp_2[slice11],
       simple_regret_stp_3[slice11],
       simple_regret_stp_4[slice11],
       simple_regret_stp_5[slice11],
       simple_regret_stp_6[slice11],
       simple_regret_stp_7[slice11],
       simple_regret_stp_8[slice11],
       simple_regret_stp_9[slice11],
       simple_regret_stp_10[slice11],
       simple_regret_stp_11[slice11],
       simple_regret_stp_12[slice11],
       simple_regret_stp_13[slice11],
       simple_regret_stp_14[slice11],
       simple_regret_stp_15[slice11],
       simple_regret_stp_16[slice11],
       simple_regret_stp_17[slice11],
       simple_regret_stp_18[slice11],
       simple_regret_stp_19[slice11],
       simple_regret_stp_20[slice11]]

gp11_results = pd.DataFrame(gp11).sort_values(by=[0], ascending=False)
stp11_results = pd.DataFrame(stp11).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp11 = np.asarray(gp11_results[4:5][0])[0]
median_gp11 = np.asarray(gp11_results[9:10][0])[0]
upper_gp11 = np.asarray(gp11_results[14:15][0])[0]

lower_stp11 = np.asarray(stp11_results[4:5][0])[0]
median_stp11 = np.asarray(stp11_results[9:10][0])[0]
upper_stp11 = np.asarray(stp11_results[14:15][0])[0]


In [74]:
# Iteration21 :

slice21 = 20

gp21 = [simple_regret_gp_1[slice21],
       simple_regret_gp_2[slice21],
       simple_regret_gp_3[slice21],
       simple_regret_gp_4[slice21],
       simple_regret_gp_5[slice21],
       simple_regret_gp_6[slice21],
       simple_regret_gp_7[slice21],
       simple_regret_gp_8[slice21],
       simple_regret_gp_9[slice21],
       simple_regret_gp_10[slice21],
       simple_regret_gp_11[slice21],
       simple_regret_gp_12[slice21],
       simple_regret_gp_13[slice21],
       simple_regret_gp_14[slice21],
       simple_regret_gp_15[slice21],
       simple_regret_gp_16[slice21],
       simple_regret_gp_17[slice21],
       simple_regret_gp_18[slice21],
       simple_regret_gp_19[slice21],
       simple_regret_gp_20[slice21]]

stp21 = [simple_regret_stp_1[slice21],
       simple_regret_stp_2[slice21],
       simple_regret_stp_3[slice21],
       simple_regret_stp_4[slice21],
       simple_regret_stp_5[slice21],
       simple_regret_stp_6[slice21],
       simple_regret_stp_7[slice21],
       simple_regret_stp_8[slice21],
       simple_regret_stp_9[slice21],
       simple_regret_stp_10[slice21],
       simple_regret_stp_11[slice21],
       simple_regret_stp_12[slice21],
       simple_regret_stp_13[slice21],
       simple_regret_stp_14[slice21],
       simple_regret_stp_15[slice21],
       simple_regret_stp_16[slice21],
       simple_regret_stp_17[slice21],
       simple_regret_stp_18[slice21],
       simple_regret_stp_19[slice21],
       simple_regret_stp_20[slice21]]

gp21_results = pd.DataFrame(gp21).sort_values(by=[0], ascending=False)
stp21_results = pd.DataFrame(stp21).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp21 = np.asarray(gp21_results[4:5][0])[0]
median_gp21 = np.asarray(gp21_results[9:10][0])[0]
upper_gp21 = np.asarray(gp21_results[14:15][0])[0]

lower_stp21 = np.asarray(stp21_results[4:5][0])[0]
median_stp21 = np.asarray(stp21_results[9:10][0])[0]
upper_stp21 = np.asarray(stp21_results[14:15][0])[0]


In [75]:
# Iteration2 :

slice2 = 1

gp2 = [simple_regret_gp_1[slice2],
       simple_regret_gp_2[slice2],
       simple_regret_gp_3[slice2],
       simple_regret_gp_4[slice2],
       simple_regret_gp_5[slice2],
       simple_regret_gp_6[slice2],
       simple_regret_gp_7[slice2],
       simple_regret_gp_8[slice2],
       simple_regret_gp_9[slice2],
       simple_regret_gp_10[slice2],
       simple_regret_gp_11[slice2],
       simple_regret_gp_12[slice2],
       simple_regret_gp_13[slice2],
       simple_regret_gp_14[slice2],
       simple_regret_gp_15[slice2],
       simple_regret_gp_16[slice2],
       simple_regret_gp_17[slice2],
       simple_regret_gp_18[slice2],
       simple_regret_gp_19[slice2],
       simple_regret_gp_20[slice2]]

stp2 = [simple_regret_stp_1[slice2],
       simple_regret_stp_2[slice2],
       simple_regret_stp_3[slice2],
       simple_regret_stp_4[slice2],
       simple_regret_stp_5[slice2],
       simple_regret_stp_6[slice2],
       simple_regret_stp_7[slice2],
       simple_regret_stp_8[slice2],
       simple_regret_stp_9[slice2],
       simple_regret_stp_10[slice2],
       simple_regret_stp_11[slice2],
       simple_regret_stp_12[slice2],
       simple_regret_stp_13[slice2],
       simple_regret_stp_14[slice2],
       simple_regret_stp_15[slice2],
       simple_regret_stp_16[slice2],
       simple_regret_stp_17[slice2],
       simple_regret_stp_18[slice2],
       simple_regret_stp_19[slice2],
       simple_regret_stp_20[slice2]]

gp2_results = pd.DataFrame(gp2).sort_values(by=[0], ascending=False)
stp2_results = pd.DataFrame(stp2).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp2 = np.asarray(gp2_results[4:5][0])[0]
median_gp2 = np.asarray(gp2_results[9:10][0])[0]
upper_gp2 = np.asarray(gp2_results[14:15][0])[0]

lower_stp2 = np.asarray(stp2_results[4:5][0])[0]
median_stp2 = np.asarray(stp2_results[9:10][0])[0]
upper_stp2 = np.asarray(stp2_results[14:15][0])[0]


In [76]:
# Iteration12 :

slice12 = 11

gp12 = [simple_regret_gp_1[slice12],
       simple_regret_gp_2[slice12],
       simple_regret_gp_3[slice12],
       simple_regret_gp_4[slice12],
       simple_regret_gp_5[slice12],
       simple_regret_gp_6[slice12],
       simple_regret_gp_7[slice12],
       simple_regret_gp_8[slice12],
       simple_regret_gp_9[slice12],
       simple_regret_gp_10[slice12],
       simple_regret_gp_11[slice12],
       simple_regret_gp_12[slice12],
       simple_regret_gp_13[slice12],
       simple_regret_gp_14[slice12],
       simple_regret_gp_15[slice12],
       simple_regret_gp_16[slice12],
       simple_regret_gp_17[slice12],
       simple_regret_gp_18[slice12],
       simple_regret_gp_19[slice12],
       simple_regret_gp_20[slice12]]

stp12 = [simple_regret_stp_1[slice12],
       simple_regret_stp_2[slice12],
       simple_regret_stp_3[slice12],
       simple_regret_stp_4[slice12],
       simple_regret_stp_5[slice12],
       simple_regret_stp_6[slice12],
       simple_regret_stp_7[slice12],
       simple_regret_stp_8[slice12],
       simple_regret_stp_9[slice12],
       simple_regret_stp_10[slice12],
       simple_regret_stp_11[slice12],
       simple_regret_stp_12[slice12],
       simple_regret_stp_13[slice12],
       simple_regret_stp_14[slice12],
       simple_regret_stp_15[slice12],
       simple_regret_stp_16[slice12],
       simple_regret_stp_17[slice12],
       simple_regret_stp_18[slice12],
       simple_regret_stp_19[slice12],
       simple_regret_stp_20[slice12]]

gp12_results = pd.DataFrame(gp12).sort_values(by=[0], ascending=False)
stp12_results = pd.DataFrame(stp12).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp12 = np.asarray(gp12_results[4:5][0])[0]
median_gp12 = np.asarray(gp12_results[9:10][0])[0]
upper_gp12 = np.asarray(gp12_results[14:15][0])[0]

lower_stp12 = np.asarray(stp12_results[4:5][0])[0]
median_stp12 = np.asarray(stp12_results[9:10][0])[0]
upper_stp12 = np.asarray(stp12_results[14:15][0])[0]


In [77]:
# Iteration3 :

slice3 = 2

gp3 = [simple_regret_gp_1[slice3],
       simple_regret_gp_2[slice3],
       simple_regret_gp_3[slice3],
       simple_regret_gp_4[slice3],
       simple_regret_gp_5[slice3],
       simple_regret_gp_6[slice3],
       simple_regret_gp_7[slice3],
       simple_regret_gp_8[slice3],
       simple_regret_gp_9[slice3],
       simple_regret_gp_10[slice3],
       simple_regret_gp_11[slice3],
       simple_regret_gp_12[slice3],
       simple_regret_gp_13[slice3],
       simple_regret_gp_14[slice3],
       simple_regret_gp_15[slice3],
       simple_regret_gp_16[slice3],
       simple_regret_gp_17[slice3],
       simple_regret_gp_18[slice3],
       simple_regret_gp_19[slice3],
       simple_regret_gp_20[slice3]]

stp3 = [simple_regret_stp_1[slice3],
       simple_regret_stp_2[slice3],
       simple_regret_stp_3[slice3],
       simple_regret_stp_4[slice3],
       simple_regret_stp_5[slice3],
       simple_regret_stp_6[slice3],
       simple_regret_stp_7[slice3],
       simple_regret_stp_8[slice3],
       simple_regret_stp_9[slice3],
       simple_regret_stp_10[slice3],
       simple_regret_stp_11[slice3],
       simple_regret_stp_12[slice3],
       simple_regret_stp_13[slice3],
       simple_regret_stp_14[slice3],
       simple_regret_stp_15[slice3],
       simple_regret_stp_16[slice3],
       simple_regret_stp_17[slice3],
       simple_regret_stp_18[slice3],
       simple_regret_stp_19[slice3],
       simple_regret_stp_20[slice3]]

gp3_results = pd.DataFrame(gp3).sort_values(by=[0], ascending=False)
stp3_results = pd.DataFrame(stp3).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp3 = np.asarray(gp3_results[4:5][0])[0]
median_gp3 = np.asarray(gp3_results[9:10][0])[0]
upper_gp3 = np.asarray(gp3_results[14:15][0])[0]

lower_stp3 = np.asarray(stp3_results[4:5][0])[0]
median_stp3 = np.asarray(stp3_results[9:10][0])[0]
upper_stp3 = np.asarray(stp3_results[14:15][0])[0]


In [78]:
# Iteration13 :

slice13 = 12

gp13 = [simple_regret_gp_1[slice13],
       simple_regret_gp_2[slice13],
       simple_regret_gp_3[slice13],
       simple_regret_gp_4[slice13],
       simple_regret_gp_5[slice13],
       simple_regret_gp_6[slice13],
       simple_regret_gp_7[slice13],
       simple_regret_gp_8[slice13],
       simple_regret_gp_9[slice13],
       simple_regret_gp_10[slice13],
       simple_regret_gp_11[slice13],
       simple_regret_gp_12[slice13],
       simple_regret_gp_13[slice13],
       simple_regret_gp_14[slice13],
       simple_regret_gp_15[slice13],
       simple_regret_gp_16[slice13],
       simple_regret_gp_17[slice13],
       simple_regret_gp_18[slice13],
       simple_regret_gp_19[slice13],
       simple_regret_gp_20[slice13]]

stp13 = [simple_regret_stp_1[slice13],
       simple_regret_stp_2[slice13],
       simple_regret_stp_3[slice13],
       simple_regret_stp_4[slice13],
       simple_regret_stp_5[slice13],
       simple_regret_stp_6[slice13],
       simple_regret_stp_7[slice13],
       simple_regret_stp_8[slice13],
       simple_regret_stp_9[slice13],
       simple_regret_stp_10[slice13],
       simple_regret_stp_11[slice13],
       simple_regret_stp_12[slice13],
       simple_regret_stp_13[slice13],
       simple_regret_stp_14[slice13],
       simple_regret_stp_15[slice13],
       simple_regret_stp_16[slice13],
       simple_regret_stp_17[slice13],
       simple_regret_stp_18[slice13],
       simple_regret_stp_19[slice13],
       simple_regret_stp_20[slice13]]

gp13_results = pd.DataFrame(gp12).sort_values(by=[0], ascending=False)
stp13_results = pd.DataFrame(stp12).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp13 = np.asarray(gp13_results[4:5][0])[0]
median_gp13 = np.asarray(gp13_results[9:10][0])[0]
upper_gp13 = np.asarray(gp13_results[14:15][0])[0]

lower_stp13 = np.asarray(stp13_results[4:5][0])[0]
median_stp13 = np.asarray(stp13_results[9:10][0])[0]
upper_stp13 = np.asarray(stp13_results[14:15][0])[0]


In [79]:
# Iteration4 :

slice4 = 3

gp4 = [simple_regret_gp_1[slice4],
       simple_regret_gp_2[slice4],
       simple_regret_gp_3[slice4],
       simple_regret_gp_4[slice4],
       simple_regret_gp_5[slice4],
       simple_regret_gp_6[slice4],
       simple_regret_gp_7[slice4],
       simple_regret_gp_8[slice4],
       simple_regret_gp_9[slice4],
       simple_regret_gp_10[slice4],
       simple_regret_gp_11[slice4],
       simple_regret_gp_12[slice4],
       simple_regret_gp_13[slice4],
       simple_regret_gp_14[slice4],
       simple_regret_gp_15[slice4],
       simple_regret_gp_16[slice4],
       simple_regret_gp_17[slice4],
       simple_regret_gp_18[slice4],
       simple_regret_gp_19[slice4],
       simple_regret_gp_20[slice4]]

stp4 = [simple_regret_stp_1[slice4],
       simple_regret_stp_2[slice4],
       simple_regret_stp_3[slice4],
       simple_regret_stp_4[slice4],
       simple_regret_stp_5[slice4],
       simple_regret_stp_6[slice4],
       simple_regret_stp_7[slice4],
       simple_regret_stp_8[slice4],
       simple_regret_stp_9[slice4],
       simple_regret_stp_10[slice4],
       simple_regret_stp_11[slice4],
       simple_regret_stp_12[slice4],
       simple_regret_stp_13[slice4],
       simple_regret_stp_14[slice4],
       simple_regret_stp_15[slice4],
       simple_regret_stp_16[slice4],
       simple_regret_stp_17[slice4],
       simple_regret_stp_18[slice4],
       simple_regret_stp_19[slice4],
       simple_regret_stp_20[slice4]]

gp4_results = pd.DataFrame(gp4).sort_values(by=[0], ascending=False)
stp4_results = pd.DataFrame(stp4).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp4 = np.asarray(gp4_results[4:5][0])[0]
median_gp4 = np.asarray(gp4_results[9:10][0])[0]
upper_gp4 = np.asarray(gp4_results[14:15][0])[0]

lower_stp4 = np.asarray(stp4_results[4:5][0])[0]
median_stp4 = np.asarray(stp4_results[9:10][0])[0]
upper_stp4 = np.asarray(stp4_results[14:15][0])[0]


In [80]:
# Iteration14 :

slice14 = 13

gp14 = [simple_regret_gp_1[slice14],
       simple_regret_gp_2[slice14],
       simple_regret_gp_3[slice14],
       simple_regret_gp_4[slice14],
       simple_regret_gp_5[slice14],
       simple_regret_gp_6[slice14],
       simple_regret_gp_7[slice14],
       simple_regret_gp_8[slice14],
       simple_regret_gp_9[slice14],
       simple_regret_gp_10[slice14],
       simple_regret_gp_11[slice14],
       simple_regret_gp_12[slice14],
       simple_regret_gp_13[slice14],
       simple_regret_gp_14[slice14],
       simple_regret_gp_15[slice14],
       simple_regret_gp_16[slice14],
       simple_regret_gp_17[slice14],
       simple_regret_gp_18[slice14],
       simple_regret_gp_19[slice14],
       simple_regret_gp_20[slice14]]

stp14 = [simple_regret_stp_1[slice14],
       simple_regret_stp_2[slice14],
       simple_regret_stp_3[slice14],
       simple_regret_stp_4[slice14],
       simple_regret_stp_5[slice14],
       simple_regret_stp_6[slice14],
       simple_regret_stp_7[slice14],
       simple_regret_stp_8[slice14],
       simple_regret_stp_9[slice14],
       simple_regret_stp_10[slice14],
       simple_regret_stp_11[slice14],
       simple_regret_stp_12[slice14],
       simple_regret_stp_13[slice14],
       simple_regret_stp_14[slice14],
       simple_regret_stp_15[slice14],
       simple_regret_stp_16[slice14],
       simple_regret_stp_17[slice14],
       simple_regret_stp_18[slice14],
       simple_regret_stp_19[slice14],
       simple_regret_stp_20[slice14]]

gp14_results = pd.DataFrame(gp14).sort_values(by=[0], ascending=False)
stp14_results = pd.DataFrame(stp14).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp14 = np.asarray(gp14_results[4:5][0])[0]
median_gp14 = np.asarray(gp14_results[9:10][0])[0]
upper_gp14 = np.asarray(gp14_results[14:15][0])[0]

lower_stp14 = np.asarray(stp14_results[4:5][0])[0]
median_stp14 = np.asarray(stp14_results[9:10][0])[0]
upper_stp14 = np.asarray(stp14_results[14:15][0])[0]


In [81]:
# Iteration5 :

slice5 = 4

gp5 = [simple_regret_gp_1[slice5],
       simple_regret_gp_2[slice5],
       simple_regret_gp_3[slice5],
       simple_regret_gp_4[slice5],
       simple_regret_gp_5[slice5],
       simple_regret_gp_6[slice5],
       simple_regret_gp_7[slice5],
       simple_regret_gp_8[slice5],
       simple_regret_gp_9[slice5],
       simple_regret_gp_10[slice5],
       simple_regret_gp_11[slice5],
       simple_regret_gp_12[slice5],
       simple_regret_gp_13[slice5],
       simple_regret_gp_14[slice5],
       simple_regret_gp_15[slice5],
       simple_regret_gp_16[slice5],
       simple_regret_gp_17[slice5],
       simple_regret_gp_18[slice5],
       simple_regret_gp_19[slice5],
       simple_regret_gp_20[slice5]]

stp5 = [simple_regret_stp_1[slice5],
       simple_regret_stp_2[slice5],
       simple_regret_stp_3[slice5],
       simple_regret_stp_4[slice5],
       simple_regret_stp_5[slice5],
       simple_regret_stp_6[slice5],
       simple_regret_stp_7[slice5],
       simple_regret_stp_8[slice5],
       simple_regret_stp_9[slice5],
       simple_regret_stp_10[slice5],
       simple_regret_stp_11[slice5],
       simple_regret_stp_12[slice5],
       simple_regret_stp_13[slice5],
       simple_regret_stp_14[slice5],
       simple_regret_stp_15[slice5],
       simple_regret_stp_16[slice5],
       simple_regret_stp_17[slice5],
       simple_regret_stp_18[slice5],
       simple_regret_stp_19[slice5],
       simple_regret_stp_20[slice5]]

gp5_results = pd.DataFrame(gp5).sort_values(by=[0], ascending=False)
stp5_results = pd.DataFrame(stp5).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp5 = np.asarray(gp5_results[4:5][0])[0]
median_gp5 = np.asarray(gp5_results[9:10][0])[0]
upper_gp5 = np.asarray(gp5_results[14:15][0])[0]

lower_stp5 = np.asarray(stp5_results[4:5][0])[0]
median_stp5 = np.asarray(stp5_results[9:10][0])[0]
upper_stp5 = np.asarray(stp5_results[14:15][0])[0]


In [82]:
# Iteration15 :

slice15 = 14

gp15 = [simple_regret_gp_1[slice15],
       simple_regret_gp_2[slice15],
       simple_regret_gp_3[slice15],
       simple_regret_gp_4[slice15],
       simple_regret_gp_5[slice15],
       simple_regret_gp_6[slice15],
       simple_regret_gp_7[slice15],
       simple_regret_gp_8[slice15],
       simple_regret_gp_9[slice15],
       simple_regret_gp_10[slice15],
       simple_regret_gp_11[slice15],
       simple_regret_gp_12[slice15],
       simple_regret_gp_13[slice15],
       simple_regret_gp_14[slice15],
       simple_regret_gp_15[slice15],
       simple_regret_gp_16[slice15],
       simple_regret_gp_17[slice15],
       simple_regret_gp_18[slice15],
       simple_regret_gp_19[slice15],
       simple_regret_gp_20[slice15]]

stp15 = [simple_regret_stp_1[slice15],
       simple_regret_stp_2[slice15],
       simple_regret_stp_3[slice15],
       simple_regret_stp_4[slice15],
       simple_regret_stp_5[slice15],
       simple_regret_stp_6[slice15],
       simple_regret_stp_7[slice15],
       simple_regret_stp_8[slice15],
       simple_regret_stp_9[slice15],
       simple_regret_stp_10[slice15],
       simple_regret_stp_11[slice15],
       simple_regret_stp_12[slice15],
       simple_regret_stp_13[slice15],
       simple_regret_stp_14[slice15],
       simple_regret_stp_15[slice15],
       simple_regret_stp_16[slice15],
       simple_regret_stp_17[slice15],
       simple_regret_stp_18[slice15],
       simple_regret_stp_19[slice15],
       simple_regret_stp_20[slice15]]

gp15_results = pd.DataFrame(gp15).sort_values(by=[0], ascending=False)
stp15_results = pd.DataFrame(stp15).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp15 = np.asarray(gp15_results[4:5][0])[0]
median_gp15 = np.asarray(gp15_results[9:10][0])[0]
upper_gp15 = np.asarray(gp15_results[14:15][0])[0]

lower_stp15 = np.asarray(stp15_results[4:5][0])[0]
median_stp15 = np.asarray(stp15_results[9:10][0])[0]
upper_stp15 = np.asarray(stp15_results[14:15][0])[0]


In [83]:
# Iteration6 :

slice6 = 5

gp6 = [simple_regret_gp_1[slice6],
       simple_regret_gp_2[slice6],
       simple_regret_gp_3[slice6],
       simple_regret_gp_4[slice6],
       simple_regret_gp_5[slice6],
       simple_regret_gp_6[slice6],
       simple_regret_gp_7[slice6],
       simple_regret_gp_8[slice6],
       simple_regret_gp_9[slice6],
       simple_regret_gp_10[slice6],
       simple_regret_gp_11[slice6],
       simple_regret_gp_12[slice6],
       simple_regret_gp_13[slice6],
       simple_regret_gp_14[slice6],
       simple_regret_gp_15[slice6],
       simple_regret_gp_16[slice6],
       simple_regret_gp_17[slice6],
       simple_regret_gp_18[slice6],
       simple_regret_gp_19[slice6],
       simple_regret_gp_20[slice6]]

stp6 = [simple_regret_stp_1[slice6],
       simple_regret_stp_2[slice6],
       simple_regret_stp_3[slice6],
       simple_regret_stp_4[slice6],
       simple_regret_stp_5[slice6],
       simple_regret_stp_6[slice6],
       simple_regret_stp_7[slice6],
       simple_regret_stp_8[slice6],
       simple_regret_stp_9[slice6],
       simple_regret_stp_10[slice6],
       simple_regret_stp_11[slice6],
       simple_regret_stp_12[slice6],
       simple_regret_stp_13[slice6],
       simple_regret_stp_14[slice6],
       simple_regret_stp_15[slice6],
       simple_regret_stp_16[slice6],
       simple_regret_stp_17[slice6],
       simple_regret_stp_18[slice6],
       simple_regret_stp_19[slice6],
       simple_regret_stp_20[slice6]]

gp6_results = pd.DataFrame(gp6).sort_values(by=[0], ascending=False)
stp6_results = pd.DataFrame(stp6).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp6 = np.asarray(gp6_results[4:5][0])[0]
median_gp6 = np.asarray(gp6_results[9:10][0])[0]
upper_gp6 = np.asarray(gp6_results[14:15][0])[0]

lower_stp6 = np.asarray(stp6_results[4:5][0])[0]
median_stp6 = np.asarray(stp6_results[9:10][0])[0]
upper_stp6 = np.asarray(stp6_results[14:15][0])[0]


In [84]:
# Iteration16 :

slice16 = 15

gp16 = [simple_regret_gp_1[slice16],
       simple_regret_gp_2[slice16],
       simple_regret_gp_3[slice16],
       simple_regret_gp_4[slice16],
       simple_regret_gp_5[slice16],
       simple_regret_gp_6[slice16],
       simple_regret_gp_7[slice16],
       simple_regret_gp_8[slice16],
       simple_regret_gp_9[slice16],
       simple_regret_gp_10[slice16],
       simple_regret_gp_11[slice16],
       simple_regret_gp_12[slice16],
       simple_regret_gp_13[slice16],
       simple_regret_gp_14[slice16],
       simple_regret_gp_15[slice16],
       simple_regret_gp_16[slice16],
       simple_regret_gp_17[slice16],
       simple_regret_gp_18[slice16],
       simple_regret_gp_19[slice16],
       simple_regret_gp_20[slice16]]

stp16 = [simple_regret_stp_1[slice16],
       simple_regret_stp_2[slice16],
       simple_regret_stp_3[slice16],
       simple_regret_stp_4[slice16],
       simple_regret_stp_5[slice16],
       simple_regret_stp_6[slice16],
       simple_regret_stp_7[slice16],
       simple_regret_stp_8[slice16],
       simple_regret_stp_9[slice16],
       simple_regret_stp_10[slice16],
       simple_regret_stp_11[slice16],
       simple_regret_stp_12[slice16],
       simple_regret_stp_13[slice16],
       simple_regret_stp_14[slice16],
       simple_regret_stp_15[slice16],
       simple_regret_stp_16[slice16],
       simple_regret_stp_17[slice16],
       simple_regret_stp_18[slice16],
       simple_regret_stp_19[slice16],
       simple_regret_stp_20[slice16]]

gp16_results = pd.DataFrame(gp16).sort_values(by=[0], ascending=False)
stp16_results = pd.DataFrame(stp16).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp16 = np.asarray(gp16_results[4:5][0])[0]
median_gp16 = np.asarray(gp16_results[9:10][0])[0]
upper_gp16 = np.asarray(gp16_results[14:15][0])[0]

lower_stp16 = np.asarray(stp16_results[4:5][0])[0]
median_stp16 = np.asarray(stp16_results[9:10][0])[0]
upper_stp16 = np.asarray(stp16_results[14:15][0])[0]


In [85]:
# Iteration7 :

slice7 = 6

gp7 = [simple_regret_gp_1[slice7],
       simple_regret_gp_2[slice7],
       simple_regret_gp_3[slice7],
       simple_regret_gp_4[slice7],
       simple_regret_gp_5[slice7],
       simple_regret_gp_6[slice7],
       simple_regret_gp_7[slice7],
       simple_regret_gp_8[slice7],
       simple_regret_gp_9[slice7],
       simple_regret_gp_10[slice7],
       simple_regret_gp_11[slice7],
       simple_regret_gp_12[slice7],
       simple_regret_gp_13[slice7],
       simple_regret_gp_14[slice7],
       simple_regret_gp_15[slice7],
       simple_regret_gp_16[slice7],
       simple_regret_gp_17[slice7],
       simple_regret_gp_18[slice7],
       simple_regret_gp_19[slice7],
       simple_regret_gp_20[slice7]]

stp7 = [simple_regret_stp_1[slice7],
       simple_regret_stp_2[slice7],
       simple_regret_stp_3[slice7],
       simple_regret_stp_4[slice7],
       simple_regret_stp_5[slice7],
       simple_regret_stp_6[slice7],
       simple_regret_stp_7[slice7],
       simple_regret_stp_8[slice7],
       simple_regret_stp_9[slice7],
       simple_regret_stp_10[slice7],
       simple_regret_stp_11[slice7],
       simple_regret_stp_12[slice7],
       simple_regret_stp_13[slice7],
       simple_regret_stp_14[slice7],
       simple_regret_stp_15[slice7],
       simple_regret_stp_16[slice7],
       simple_regret_stp_17[slice7],
       simple_regret_stp_18[slice7],
       simple_regret_stp_19[slice7],
       simple_regret_stp_20[slice7]]

gp7_results = pd.DataFrame(gp7).sort_values(by=[0], ascending=False)
stp7_results = pd.DataFrame(stp7).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp7 = np.asarray(gp7_results[4:5][0])[0]
median_gp7 = np.asarray(gp7_results[9:10][0])[0]
upper_gp7 = np.asarray(gp7_results[14:15][0])[0]

lower_stp7 = np.asarray(stp7_results[4:5][0])[0]
median_stp7 = np.asarray(stp7_results[9:10][0])[0]
upper_stp7 = np.asarray(stp7_results[14:15][0])[0]


In [86]:
# Iteration17 :

slice17 = 16

gp17 = [simple_regret_gp_1[slice17],
       simple_regret_gp_2[slice17],
       simple_regret_gp_3[slice17],
       simple_regret_gp_4[slice17],
       simple_regret_gp_5[slice17],
       simple_regret_gp_6[slice17],
       simple_regret_gp_7[slice17],
       simple_regret_gp_8[slice17],
       simple_regret_gp_9[slice17],
       simple_regret_gp_10[slice17],
       simple_regret_gp_11[slice17],
       simple_regret_gp_12[slice17],
       simple_regret_gp_13[slice17],
       simple_regret_gp_14[slice17],
       simple_regret_gp_15[slice17],
       simple_regret_gp_16[slice17],
       simple_regret_gp_17[slice17],
       simple_regret_gp_18[slice17],
       simple_regret_gp_19[slice17],
       simple_regret_gp_20[slice17]]

stp17 = [simple_regret_stp_1[slice17],
       simple_regret_stp_2[slice17],
       simple_regret_stp_3[slice17],
       simple_regret_stp_4[slice17],
       simple_regret_stp_5[slice17],
       simple_regret_stp_6[slice17],
       simple_regret_stp_7[slice17],
       simple_regret_stp_8[slice17],
       simple_regret_stp_9[slice17],
       simple_regret_stp_10[slice17],
       simple_regret_stp_11[slice17],
       simple_regret_stp_12[slice17],
       simple_regret_stp_13[slice17],
       simple_regret_stp_14[slice17],
       simple_regret_stp_15[slice17],
       simple_regret_stp_16[slice17],
       simple_regret_stp_17[slice17],
       simple_regret_stp_18[slice17],
       simple_regret_stp_19[slice17],
       simple_regret_stp_20[slice17]]

gp17_results = pd.DataFrame(gp17).sort_values(by=[0], ascending=False)
stp17_results = pd.DataFrame(stp17).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp17 = np.asarray(gp17_results[4:5][0])[0]
median_gp17 = np.asarray(gp17_results[9:10][0])[0]
upper_gp17 = np.asarray(gp17_results[14:15][0])[0]

lower_stp17 = np.asarray(stp17_results[4:5][0])[0]
median_stp17 = np.asarray(stp17_results[9:10][0])[0]
upper_stp17 = np.asarray(stp17_results[14:15][0])[0]


In [87]:
# Iteration8 :

slice8 = 7

gp8 = [simple_regret_gp_1[slice8],
       simple_regret_gp_2[slice8],
       simple_regret_gp_3[slice8],
       simple_regret_gp_4[slice8],
       simple_regret_gp_5[slice8],
       simple_regret_gp_6[slice8],
       simple_regret_gp_7[slice8],
       simple_regret_gp_8[slice8],
       simple_regret_gp_9[slice8],
       simple_regret_gp_10[slice8],
       simple_regret_gp_11[slice8],
       simple_regret_gp_12[slice8],
       simple_regret_gp_13[slice8],
       simple_regret_gp_14[slice8],
       simple_regret_gp_15[slice8],
       simple_regret_gp_16[slice8],
       simple_regret_gp_17[slice8],
       simple_regret_gp_18[slice8],
       simple_regret_gp_19[slice8],
       simple_regret_gp_20[slice8]]

stp8 = [simple_regret_stp_1[slice8],
       simple_regret_stp_2[slice8],
       simple_regret_stp_3[slice8],
       simple_regret_stp_4[slice8],
       simple_regret_stp_5[slice8],
       simple_regret_stp_6[slice8],
       simple_regret_stp_7[slice8],
       simple_regret_stp_8[slice8],
       simple_regret_stp_9[slice8],
       simple_regret_stp_10[slice8],
       simple_regret_stp_11[slice8],
       simple_regret_stp_12[slice8],
       simple_regret_stp_13[slice8],
       simple_regret_stp_14[slice8],
       simple_regret_stp_15[slice8],
       simple_regret_stp_16[slice8],
       simple_regret_stp_17[slice8],
       simple_regret_stp_18[slice8],
       simple_regret_stp_19[slice8],
       simple_regret_stp_20[slice8]]

gp8_results = pd.DataFrame(gp8).sort_values(by=[0], ascending=False)
stp8_results = pd.DataFrame(stp8).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp8 = np.asarray(gp8_results[4:5][0])[0]
median_gp8 = np.asarray(gp8_results[9:10][0])[0]
upper_gp8 = np.asarray(gp8_results[14:15][0])[0]

lower_stp8 = np.asarray(stp8_results[4:5][0])[0]
median_stp8 = np.asarray(stp8_results[9:10][0])[0]
upper_stp8 = np.asarray(stp8_results[14:15][0])[0]


In [88]:
# Iteration18 :

slice18 = 17

gp18 = [simple_regret_gp_1[slice18],
       simple_regret_gp_2[slice18],
       simple_regret_gp_3[slice18],
       simple_regret_gp_4[slice18],
       simple_regret_gp_5[slice18],
       simple_regret_gp_6[slice18],
       simple_regret_gp_7[slice18],
       simple_regret_gp_8[slice18],
       simple_regret_gp_9[slice18],
       simple_regret_gp_10[slice18],
       simple_regret_gp_11[slice18],
       simple_regret_gp_12[slice18],
       simple_regret_gp_13[slice18],
       simple_regret_gp_14[slice18],
       simple_regret_gp_15[slice18],
       simple_regret_gp_16[slice18],
       simple_regret_gp_17[slice18],
       simple_regret_gp_18[slice18],
       simple_regret_gp_19[slice18],
       simple_regret_gp_20[slice18]]

stp18 = [simple_regret_stp_1[slice18],
       simple_regret_stp_2[slice18],
       simple_regret_stp_3[slice18],
       simple_regret_stp_4[slice18],
       simple_regret_stp_5[slice18],
       simple_regret_stp_6[slice18],
       simple_regret_stp_7[slice18],
       simple_regret_stp_8[slice18],
       simple_regret_stp_9[slice18],
       simple_regret_stp_10[slice18],
       simple_regret_stp_11[slice18],
       simple_regret_stp_12[slice18],
       simple_regret_stp_13[slice18],
       simple_regret_stp_14[slice18],
       simple_regret_stp_15[slice18],
       simple_regret_stp_16[slice18],
       simple_regret_stp_17[slice18],
       simple_regret_stp_18[slice18],
       simple_regret_stp_19[slice18],
       simple_regret_stp_20[slice18]]

gp18_results = pd.DataFrame(gp18).sort_values(by=[0], ascending=False)
stp18_results = pd.DataFrame(stp18).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp18 = np.asarray(gp18_results[4:5][0])[0]
median_gp18 = np.asarray(gp18_results[9:10][0])[0]
upper_gp18 = np.asarray(gp18_results[14:15][0])[0]

lower_stp18 = np.asarray(stp18_results[4:5][0])[0]
median_stp18 = np.asarray(stp18_results[9:10][0])[0]
upper_stp18 = np.asarray(stp18_results[14:15][0])[0]


In [89]:
# Iteration9 :

slice9 = 8

gp9 = [simple_regret_gp_1[slice9],
       simple_regret_gp_2[slice9],
       simple_regret_gp_3[slice9],
       simple_regret_gp_4[slice9],
       simple_regret_gp_5[slice9],
       simple_regret_gp_6[slice9],
       simple_regret_gp_7[slice9],
       simple_regret_gp_8[slice9],
       simple_regret_gp_9[slice9],
       simple_regret_gp_10[slice9],
       simple_regret_gp_11[slice9],
       simple_regret_gp_12[slice9],
       simple_regret_gp_13[slice9],
       simple_regret_gp_14[slice9],
       simple_regret_gp_15[slice9],
       simple_regret_gp_16[slice9],
       simple_regret_gp_17[slice9],
       simple_regret_gp_18[slice9],
       simple_regret_gp_19[slice9],
       simple_regret_gp_20[slice9]]

stp9 = [simple_regret_stp_1[slice9],
       simple_regret_stp_2[slice9],
       simple_regret_stp_3[slice9],
       simple_regret_stp_4[slice9],
       simple_regret_stp_5[slice9],
       simple_regret_stp_6[slice9],
       simple_regret_stp_7[slice9],
       simple_regret_stp_8[slice9],
       simple_regret_stp_9[slice9],
       simple_regret_stp_10[slice9],
       simple_regret_stp_11[slice9],
       simple_regret_stp_12[slice9],
       simple_regret_stp_13[slice9],
       simple_regret_stp_14[slice9],
       simple_regret_stp_15[slice9],
       simple_regret_stp_16[slice9],
       simple_regret_stp_17[slice9],
       simple_regret_stp_18[slice9],
       simple_regret_stp_19[slice9],
       simple_regret_stp_20[slice9]]

gp9_results = pd.DataFrame(gp9).sort_values(by=[0], ascending=False)
stp9_results = pd.DataFrame(stp9).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp9 = np.asarray(gp9_results[4:5][0])[0]
median_gp9 = np.asarray(gp9_results[9:10][0])[0]
upper_gp9 = np.asarray(gp9_results[14:15][0])[0]

lower_stp9 = np.asarray(stp9_results[4:5][0])[0]
median_stp9 = np.asarray(stp9_results[9:10][0])[0]
upper_stp9 = np.asarray(stp9_results[14:15][0])[0]


In [90]:
# Iteration19 :

slice19 = 18

gp19 = [simple_regret_gp_1[slice19],
       simple_regret_gp_2[slice19],
       simple_regret_gp_3[slice19],
       simple_regret_gp_4[slice19],
       simple_regret_gp_5[slice19],
       simple_regret_gp_6[slice19],
       simple_regret_gp_7[slice19],
       simple_regret_gp_8[slice19],
       simple_regret_gp_9[slice19],
       simple_regret_gp_10[slice19],
       simple_regret_gp_11[slice19],
       simple_regret_gp_12[slice19],
       simple_regret_gp_13[slice19],
       simple_regret_gp_14[slice19],
       simple_regret_gp_15[slice19],
       simple_regret_gp_16[slice19],
       simple_regret_gp_17[slice19],
       simple_regret_gp_18[slice19],
       simple_regret_gp_19[slice19],
       simple_regret_gp_20[slice19]]

stp19 = [simple_regret_stp_1[slice19],
       simple_regret_stp_2[slice19],
       simple_regret_stp_3[slice19],
       simple_regret_stp_4[slice19],
       simple_regret_stp_5[slice19],
       simple_regret_stp_6[slice19],
       simple_regret_stp_7[slice19],
       simple_regret_stp_8[slice19],
       simple_regret_stp_9[slice19],
       simple_regret_stp_10[slice19],
       simple_regret_stp_11[slice19],
       simple_regret_stp_12[slice19],
       simple_regret_stp_13[slice19],
       simple_regret_stp_14[slice19],
       simple_regret_stp_15[slice19],
       simple_regret_stp_16[slice19],
       simple_regret_stp_17[slice19],
       simple_regret_stp_18[slice19],
       simple_regret_stp_19[slice19],
       simple_regret_stp_20[slice19]]

gp19_results = pd.DataFrame(gp19).sort_values(by=[0], ascending=False)
stp19_results = pd.DataFrame(stp19).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp19 = np.asarray(gp19_results[4:5][0])[0]
median_gp19 = np.asarray(gp19_results[9:10][0])[0]
upper_gp19 = np.asarray(gp19_results[14:15][0])[0]

lower_stp19 = np.asarray(stp19_results[4:5][0])[0]
median_stp19 = np.asarray(stp19_results[9:10][0])[0]
upper_stp19 = np.asarray(stp19_results[14:15][0])[0]


In [91]:
# Iteration10 :

slice10 = 9

gp10 = [simple_regret_gp_1[slice10],
       simple_regret_gp_2[slice10],
       simple_regret_gp_3[slice10],
       simple_regret_gp_4[slice10],
       simple_regret_gp_5[slice10],
       simple_regret_gp_6[slice10],
       simple_regret_gp_7[slice10],
       simple_regret_gp_8[slice10],
       simple_regret_gp_9[slice10],
       simple_regret_gp_10[slice10],
       simple_regret_gp_11[slice10],
       simple_regret_gp_12[slice10],
       simple_regret_gp_13[slice10],
       simple_regret_gp_14[slice10],
       simple_regret_gp_15[slice10],
       simple_regret_gp_16[slice10],
       simple_regret_gp_17[slice10],
       simple_regret_gp_18[slice10],
       simple_regret_gp_19[slice10],
       simple_regret_gp_20[slice10]]

stp10 = [simple_regret_stp_1[slice10],
       simple_regret_stp_2[slice10],
       simple_regret_stp_3[slice10],
       simple_regret_stp_4[slice10],
       simple_regret_stp_5[slice10],
       simple_regret_stp_6[slice10],
       simple_regret_stp_7[slice10],
       simple_regret_stp_8[slice10],
       simple_regret_stp_9[slice10],
       simple_regret_stp_10[slice10],
       simple_regret_stp_11[slice10],
       simple_regret_stp_12[slice10],
       simple_regret_stp_13[slice10],
       simple_regret_stp_14[slice10],
       simple_regret_stp_15[slice10],
       simple_regret_stp_16[slice10],
       simple_regret_stp_17[slice10],
       simple_regret_stp_18[slice10],
       simple_regret_stp_19[slice10],
       simple_regret_stp_20[slice10]]

gp10_results = pd.DataFrame(gp10).sort_values(by=[0], ascending=False)
stp10_results = pd.DataFrame(stp10).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp10 = np.asarray(gp10_results[4:5][0])[0]
median_gp10 = np.asarray(gp10_results[9:10][0])[0]
upper_gp10 = np.asarray(gp10_results[14:15][0])[0]

lower_stp10 = np.asarray(stp10_results[4:5][0])[0]
median_stp10 = np.asarray(stp10_results[9:10][0])[0]
upper_stp10 = np.asarray(stp10_results[14:15][0])[0]


In [92]:
# Iteration20 :

slice20 = 19

gp20 = [simple_regret_gp_1[slice20],
       simple_regret_gp_2[slice20],
       simple_regret_gp_3[slice20],
       simple_regret_gp_4[slice20],
       simple_regret_gp_5[slice20],
       simple_regret_gp_6[slice20],
       simple_regret_gp_7[slice20],
       simple_regret_gp_8[slice20],
       simple_regret_gp_9[slice20],
       simple_regret_gp_10[slice20],
       simple_regret_gp_11[slice20],
       simple_regret_gp_12[slice20],
       simple_regret_gp_13[slice20],
       simple_regret_gp_14[slice20],
       simple_regret_gp_15[slice20],
       simple_regret_gp_16[slice20],
       simple_regret_gp_17[slice20],
       simple_regret_gp_18[slice20],
       simple_regret_gp_19[slice20],
       simple_regret_gp_20[slice20]]

stp20 = [simple_regret_stp_1[slice20],
       simple_regret_stp_2[slice20],
       simple_regret_stp_3[slice20],
       simple_regret_stp_4[slice20],
       simple_regret_stp_5[slice20],
       simple_regret_stp_6[slice20],
       simple_regret_stp_7[slice20],
       simple_regret_stp_8[slice20],
       simple_regret_stp_9[slice20],
       simple_regret_stp_10[slice20],
       simple_regret_stp_11[slice20],
       simple_regret_stp_12[slice20],
       simple_regret_stp_13[slice20],
       simple_regret_stp_14[slice20],
       simple_regret_stp_15[slice20],
       simple_regret_stp_16[slice20],
       simple_regret_stp_17[slice20],
       simple_regret_stp_18[slice20],
       simple_regret_stp_19[slice20],
       simple_regret_stp_20[slice20]]

gp20_results = pd.DataFrame(gp20).sort_values(by=[0], ascending=False)
stp20_results = pd.DataFrame(stp20).sort_values(by=[0], ascending=False)

### Best training regret minimization IQR - gp:
lower_gp20 = np.asarray(gp20_results[4:5][0])[0]
median_gp20 = np.asarray(gp20_results[9:10][0])[0]
upper_gp20 = np.asarray(gp20_results[14:15][0])[0]

lower_stp20 = np.asarray(stp20_results[4:5][0])[0]
median_stp20 = np.asarray(stp20_results[9:10][0])[0]
upper_stp20 = np.asarray(stp20_results[14:15][0])[0]


In [93]:
### Summarise arrays: 'GP'

lower_gp = [lower_gp1,
            lower_gp2,
            lower_gp3,
            lower_gp4,
            lower_gp5,
            lower_gp6,
            lower_gp7,
            lower_gp8,
            lower_gp9,
            lower_gp10,
            lower_gp11,
            lower_gp12,
            lower_gp13,
            lower_gp14,
            lower_gp15,
            lower_gp16,
            lower_gp17,
            lower_gp18,
            lower_gp19,
            lower_gp20,
            lower_gp21]

median_gp = [median_gp1,
            median_gp2,
            median_gp3,
            median_gp4,
            median_gp5,
            median_gp6,
            median_gp7,
            median_gp8,
            median_gp9,
            median_gp10,
            median_gp11,
            median_gp12,
            median_gp13,
            median_gp14,
            median_gp15,
            median_gp16,
            median_gp17,
            median_gp18,
            median_gp19,
            median_gp20,
            median_gp21]

upper_gp = [upper_gp1,
            upper_gp2,
            upper_gp3,
            upper_gp4,
            upper_gp5,
            upper_gp6,
            upper_gp7,
            upper_gp8,
            upper_gp9,
            upper_gp10,
            upper_gp11,
            upper_gp12,
            upper_gp13,
            upper_gp14,
            upper_gp15,
            upper_gp16,
            upper_gp17,
            upper_gp18,
            upper_gp19,
            upper_gp20,
            upper_gp21]


In [94]:
### Summarise arrays: 'STP'

lower_stp = [lower_stp1,
            lower_stp2,
            lower_stp3,
            lower_stp4,
            lower_stp5,
            lower_stp6,
            lower_stp7,
            lower_stp8,
            lower_stp9,
            lower_stp10,
            lower_stp11,
            lower_stp12,
            lower_stp13,
            lower_stp14,
            lower_stp15,
            lower_stp16,
            lower_stp17,
            lower_stp18,
            lower_stp19,
            lower_stp20,
            lower_stp21]

median_stp = [median_stp1,
            median_stp2,
            median_stp3,
            median_stp4,
            median_stp5,
            median_stp6,
            median_stp7,
            median_stp8,
            median_stp9,
            median_stp10,
            median_stp11,
            median_stp12,
            median_stp13,
            median_stp14,
            median_stp15,
            median_stp16,
            median_stp17,
            median_stp18,
            median_stp19,
            median_stp20,
            median_stp21]

upper_stp = [upper_stp1,
            upper_stp2,
            upper_stp3,
            upper_stp4,
            upper_stp5,
            upper_stp6,
            upper_stp7,
            upper_stp8,
            upper_stp9,
            upper_stp10,
            upper_stp11,
            upper_stp12,
            upper_stp13,
            upper_stp14,
            upper_stp15,
            upper_stp16,
            upper_stp17,
            upper_stp18,
            upper_stp19,
            upper_stp20,
            upper_stp21]


In [95]:
### Visualise!

title = obj_func

plt.figure()

plt.plot(median_gp, color = 'Red')
plt.plot(median_stp, color = 'Blue')

xstar = np.arange(0, max_iter+1, step=1)
plt.fill_between(xstar, lower_gp, upper_gp, facecolor = 'Red', alpha=0.4, label='GP ERM Regret: IQR')
plt.fill_between(xstar, lower_stp, upper_stp, facecolor = 'Blue', alpha=0.4, label='STP ERM Regret: IQR ' r'($\nu$' ' = {})'.format(df))

plt.title(title, weight = 'bold')
plt.xlabel('(Post-initialisation) iteration $\it{k}$', weight = 'bold') # x-axis label
plt.ylabel('log(Regret)', weight = 'bold') # y-axis label
plt.legend(loc=0) # add plot legend

### Make the x-ticks integers, not floats:
count = len(xstar)
plt.xticks(np.arange(count), np.arange(0, count))
plt.show() #visualise!
